In [12]:
from docx import Document
import re
from collections import defaultdict
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger le modèle NLP pour le résumé automatique
nlp = spacy.load("en_core_web_sm")

def extract_themes_summaries(doc_path):
    doc = Document(doc_path)
    
    # Vérification du nombre de paragraphes
    if len(doc.paragraphs) == 0:
        print("Aucun paragraphe détecté dans le document.")
        return {}
    
    themes = defaultdict(list)
    current_theme = "Autre"  # Thème par défaut
    
    for para in doc.paragraphs:
        text = para.text.strip()
        
        # Affichage des styles pour diagnostic
        print(f"Texte: {text[:50]}... | Style: {para.style.name}")
        
        # Détection améliorée des thématiques (inclut titres sans styles spécifiques)
        if (
            para.style.name in ['Heading 1', 'Heading 2', 'Heading 3'] or
            text.isupper() or
            re.match(r'^(\d+\.\s+|[A-Z][a-z]+\s+[A-Z][a-z]+)', text) or  # Détection de numérotation et titres courants
            len(text.split()) < 6  # Texte court = probablement un titre
        ):
            current_theme = text
            themes[current_theme] = []
            continue
        
        # Ajout du texte sous la thématique courante
        if current_theme in themes:
            themes[current_theme].append(text)
    
    # Vérification des thématiques extraites
    print("Thématiques détectées:", themes.keys())
    
    # Génération d'un résumé optimisé par NLP et TextRank
    summaries = {}
    for theme, texts in themes.items():
        full_text = " ".join(texts)
        if not full_text.strip():
            summaries[theme] = "Aucune information disponible."
            continue
        
        doc_nlp = nlp(full_text)
        sentences = [sent.text for sent in doc_nlp.sents]
        
        if len(sentences) > 3:
            # Utilisation de TF-IDF pour identifier les phrases les plus informatives
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(sentences)
            sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
            top_sentence_indices = sentence_scores.argsort()[-3:][::-1]
            summary_sentences = [sentences[i] for i in sorted(top_sentence_indices)]
            summaries[theme] = " ".join(summary_sentences)
        else:
            summaries[theme] = full_text if full_text else "Aucune information disponible."
    
    return summaries

# Exemple d'utilisation
doc_path = "../data/projet_1/PU_P01_AAP01.docx"  # Remplace par ton fichier
theme_summaries = extract_themes_summaries(doc_path)

# Affichage formaté des résultats
for theme, summary in theme_summaries.items():
    print(f"\n### {theme} ###")
    print(f"Résumé: {summary}\n")

Texte: * Please fill it out briefly with a total length l... | Style: Normal
Texte: ... | Style: Normal
Texte: For Standard Grant... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: Project objective　(Explain which GBF goal project ... | Style: Normal
Texte: Note: Please specify which target(s) your project ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: 2.  Project implementation plan... | Style: Normal
Texte: (Please describe the projects by year and items.)... | Style: Normal
Texte: Note; For project, please describe not only fiscal... | Style: Normal
Texte: ... | Style: Normal
Texte: Applying FY... | Style: List Paragraph
Texte: FY(s) before applying FY... | Style: List Paragraph
Texte: FY(s) after applying FY... | Style: List Paragraph
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: 3．Expected concrete

In [1]:
import requests

response = requests.post(
    "http://127.0.0.1:11435/api/generate",
    json={"model": "mistral", "prompt": "Bonjour, comment vas-tu ?", "stream": False},
)

print(response.json())


{'error': "model 'mistral' not found"}


In [14]:
from docx import Document
import re
from collections import defaultdict
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger le modèle NLP pour le résumé automatique
nlp = spacy.load("en_core_web_sm")

def is_valid_theme(text):
    """ Vérifie si un texte est un titre valide """
    invalid_starts = ["Note", "・", "Attach", "Please", "1000 caractères restants", "Ce champ est requis"]
    if len(text.split()) < 3 or any(text.startswith(prefix) for prefix in invalid_starts):
        return False
    return True

def extract_themes_summaries(doc_paths):
    all_summaries = {}
    
    for doc_path in doc_paths:
        doc = Document(doc_path)
        
        # Vérification du nombre de paragraphes
        if len(doc.paragraphs) == 0:
            print(f"Aucun paragraphe détecté dans {doc_path}.")
            continue
        
        themes = defaultdict(list)
        current_theme = "Autre"  # Thème par défaut
        
        for para in doc.paragraphs:
            text = para.text.strip()
            
            # Affichage des styles pour diagnostic
            print(f"Texte: {text[:50]}... | Style: {para.style.name}")
            
            # Détection améliorée des thématiques
            if (
                para.style.name in ['Heading 1', 'Heading 2', 'Heading 3'] or
                text.isupper() or
                re.match(r'^(\d+\.\s+|[A-Z][a-z]+\s+[A-Z][a-z]+)', text) or  # Détection numérotation et titres
                len(text.split()) < 6  # Texte court = probablement un titre
            ) and is_valid_theme(text):
                current_theme = text
                themes[current_theme] = []
                continue
            
            # Ajout du texte sous la thématique courante
            if current_theme in themes:
                themes[current_theme].append(text)
        
        # Vérification des thématiques extraites
        print(f"Thématiques détectées dans {doc_path}: {themes.keys()}")
        
        # Génération d'un résumé optimisé par NLP et TextRank
        summaries = {}
        for theme, texts in themes.items():
            full_text = " ".join(texts)
            if not full_text.strip():
                summaries[theme] = "Aucune information disponible."
                continue
            
            doc_nlp = nlp(full_text)
            sentences = [sent.text for sent in doc_nlp.sents]
            
            if len(sentences) > 3:
                # Utilisation de TF-IDF pour identifier les phrases les plus informatives
                vectorizer = TfidfVectorizer()
                tfidf_matrix = vectorizer.fit_transform(sentences)
                sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
                top_sentence_indices = sentence_scores.argsort()[-3:][::-1]
                summary_sentences = [sentences[i] for i in sorted(top_sentence_indices)]
                summaries[theme] = " ".join(summary_sentences)
            else:
                # Sélectionner la phrase la plus longue si le résumé est vide
                longest_sentence = max(sentences, key=len, default="Aucune information disponible.")
                summaries[theme] = longest_sentence
        
        all_summaries[doc_path] = summaries
    
    return all_summaries
# Exemple d'utilisation
doc_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
theme_summaries = extract_themes_summaries(doc_paths)

# Affichage formaté des résultats
for doc, summaries in theme_summaries.items():
    print(f"\n===== Résumé pour {doc} =====")
    for theme, summary in summaries.items():
        print(f"\n### {theme} ###")
        print(f"Résumé: {summary}\n")

Texte: * Please fill it out briefly with a total length l... | Style: Normal
Texte: ... | Style: Normal
Texte: For Standard Grant... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: Project objective　(Explain which GBF goal project ... | Style: Normal
Texte: Note: Please specify which target(s) your project ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: 2.  Project implementation plan... | Style: Normal
Texte: (Please describe the projects by year and items.)... | Style: Normal
Texte: Note; For project, please describe not only fiscal... | Style: Normal
Texte: ... | Style: Normal
Texte: Applying FY... | Style: List Paragraph
Texte: FY(s) before applying FY... | Style: List Paragraph
Texte: FY(s) after applying FY... | Style: List Paragraph
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: ... | Style: Normal
Texte: 3．Expected concrete

In [2]:
import requests
from docx import Document
import re
from collections import defaultdict

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def is_valid_theme(text):
    """ Vérifie si un texte est un titre valide """
    invalid_starts = ["Note", "・", "Attach", "Please", "1000 caractères restants", "Ce champ est requis"]
    return len(text.split()) >= 3 and not any(text.startswith(prefix) for prefix in invalid_starts)

def ask_mixtral(prompt):
    """ Envoie une requête au modèle Mixtral pour classifier et résumer le texte """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

def extract_themes_summaries(doc_paths):
    all_summaries = {}

    for doc_path in doc_paths:
        doc = Document(doc_path)
        if len(doc.paragraphs) == 0:
            print(f"Aucun paragraphe détecté dans {doc_path}.")
            continue
        
        themes = defaultdict(list)
        current_theme = "Autre"  # Thème par défaut

        for para in doc.paragraphs:
            text = para.text.strip()
            if (
                para.style.name in ['Heading 1', 'Heading 2', 'Heading 3'] or
                text.isupper() or
                re.match(r'^(\d+\.\s+|[A-Z][a-z]+\s+[A-Z][a-z]+)', text) or
                len(text.split()) < 6
            ) and is_valid_theme(text):
                current_theme = text
                themes[current_theme] = []
                continue

            if current_theme in themes:
                themes[current_theme].append(text)

        print(f"Thématiques détectées dans {doc_path}: {themes.keys()}")

        summaries = {}
        for theme, texts in themes.items():
            full_text = " ".join(texts)
            if not full_text.strip():
                summaries[theme] = "Aucune information disponible."
                continue

            prompt = f"Catégorise et résume la section suivante d'un document d'appel à projet:\n\n{full_text}"
            mixtral_response = ask_mixtral(prompt)
            summaries[theme] = mixtral_response
        
        all_summaries[doc_path] = summaries

    return all_summaries

# Exemple d'utilisation avec tes fichiers
doc_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
theme_summaries = extract_themes_summaries(doc_paths)

# Affichage formaté des résultats
for doc, summaries in theme_summaries.items():
    print(f"\n===== Résumé pour {doc} =====")
    for theme, summary in summaries.items():
        print(f"\n### {theme} ###")
        print(f"Résumé et Classification: {summary}\n")


Thématiques détectées dans ../data/projet_1/PU_P01_AAP01.docx: dict_keys(['For Standard Grant', '2.  Project implementation plan', 'FY(s) before applying FY', 'FY(s) after applying FY', '3．Expected concrete activity results', '（１）\tApplying FY', '（２）\tFY(s) before applying FY', '（３）\tFY(s) after applying FY', '4.  Activity schedule', '4. Activity schedule', '5. Income and Expenditure Budget plan', '6. The name and contact information of the experts outside of applicant’s who will provive advice and guidance for the project implementation .', '7. Local approvals (if you need local government approvals or agreements with local residents, please describe the contents)', '8. Partners (if you partner with a local NGO or other international organization on the projects, please list the names of the organizations)', '9. Japanese Introducer (Name and contact information)', 'Form of the organization'])
Thématiques détectées dans ../data/projet_1/PU_P01_AAP02.docx: dict_keys(['Description du pro

In [4]:
import requests
from docx import Document

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte par sections d'un document Word """
    doc = Document(doc_path)
    sections = {}
    current_section = "Introduction"  # Nom générique si aucun titre n'est trouvé

    for para in doc.paragraphs:
        text = para.text.strip()
        if text and len(text.split()) < 10:  # Supposons que les titres sont courts
            current_section = text
            sections[current_section] = []
        elif text:
            sections[current_section].append(text)

    return {sec: " ".join(content) for sec, content in sections.items()}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

# Charger le PU_P01_PP pour connaître les sections requises
pp_sections = extract_text_by_sections("../data/projet_1/PU_P01_PP.docx")

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    # Construire la requête pour Mixtral
    prompt = f"""
    Voici les sections attendues dans un appel à projets valide:
    {list(pp_sections.keys())}

    Voici les sections trouvées dans le document {aap_path}:
    {list(aap_sections.keys())}

    Vérifie si toutes les sections requises sont présentes.
    - Si elles sont toutes présentes, réponds simplement : "✅ Toutes les informations requises sont présentes."
    - Sinon, liste les sections manquantes en format simple : "❌ Manque les sections : [Liste]".
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats
for doc, result in results.items():
    print(f"\n===== Vérification pour {doc} =====")
    print(result)


KeyError: 'Introduction'

In [5]:
import requests
from docx import Document

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte par sections d'un document Word en structurant les parties détectées """
    doc = Document(doc_path)
    sections = {}
    current_section = "Section Générique"  # Ajout d'une section par défaut

    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            # Détecter les titres comme les sections (basé sur la longueur et la mise en forme)
            if len(text.split()) < 10 and text[-1] not in ".!?":
                current_section = text
                sections[current_section] = []
            else:
                if current_section not in sections:
                    sections[current_section] = []  # Évite les erreurs KeyError
                sections[current_section].append(text)

    return {sec: " ".join(content) for sec, content in sections.items() if content}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

# Charger le PU_P01_PP pour connaître les sections requises
pp_sections = extract_text_by_sections("../data/projet_1/PU_P01_PP.docx")

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    # Construire la requête pour Mixtral
    prompt = f"""
    Voici les sections attendues dans un appel à projets valide:
    {list(pp_sections.keys())}

    Voici les sections trouvées dans le document {aap_path}:
    {list(aap_sections.keys())}

    Vérifie si toutes les sections requises sont présentes.
    - Si elles sont toutes présentes, réponds simplement : "✅ Toutes les informations requises sont présentes."
    - Sinon, liste les sections manquantes en format simple : "❌ Manque les sections : [Liste]".
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats
for doc, result in results.items():
    print(f"\n===== Vérification pour {doc} =====")
    print(result)



===== Vérification pour ../data/projet_1/PU_P01_AAP01.docx =====
❌ Manque les sections : ['Brief project description', 'Name of the organization', 'Address', 'Restoring mangrove ecosystems, especially in the Mahakam Delta', 'Some activities conducted by POKJA Pesisir', 'Geographic and socio-economic context', 'Environmental context', 'Table 1. Critical Criteria of Mahakam Delta Mangrove', 'Biodiversity issues', 'Institutional Context', 'The Movement of Indonesian New Capital', 'Aquaculture industry', 'Demography', 'Other issues', 'Strategy & theory of change', 'Lack of alternative sustainable livelihood in coastal area;', 'Environmental awareness;', 'Beneficiaries', 'Teachers (primary school teachers)', 'Village officials', 'Location', '1.1.1. -Workshop on Environmental Conservation for Primary School Teachers', '1.2.1 -Encouraging/assisting schools towards adiwiyata in Mahakam Delta', '1.3.1. Waste Management Campaign', 'Target: 100 persons local community involved', 'Health approach

In [7]:
import requests
from docx import Document

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte des sections d'un document Word """
    doc = Document(doc_path)
    sections = {}
    current_section = "Section Générique"

    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            if len(text.split()) < 10 and text[-1] not in ".!?":
                current_section = text
                sections[current_section] = []
            else:
                if current_section not in sections:
                    sections[current_section] = []
                sections[current_section].append(text)

    return {sec: " ".join(content) for sec, content in sections.items() if content}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse et compréhension du texte """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

# Charger les sections attendues du PU_P01_PP
pp_sections = extract_text_by_sections("../data/projet_1/PU_P01_PP.docx")

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    # Construire une requête LLM plus avancée
    prompt = f"""
    Voici les sections attendues dans un appel à projet valide (PU_P01_PP) :
    {list(pp_sections.keys())}

    Voici les sections trouvées dans {aap_path} :
    {list(aap_sections.keys())}

    **Tâche** :
    - Vérifie si chaque section attendue est bien couverte dans le document.
    - Même si le titre diffère, vérifie si le contenu correspondant est présent sous une autre section.
    - Si tout est bien couvert, réponds : ✅ Toutes les informations requises sont présentes.
    - Si des parties sont absentes ou incomplètes, liste clairement les sections manquantes : "❌ Manque les sections : [Liste]".
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats
for doc, result in results.items():
    print(f"\n===== Vérification pour {doc} =====")
    print(result)



===== Vérification pour ../data/projet_1/PU_P01_AAP01.docx =====
❌ Manque les sections : ['Brief project description', 'Name of the organization: Pokja Pesisir', 'Website: http://pokjapesisir.id/', 'Some activities conducted by POKJA Pesisir:', 'Geographic and socio-economic context', 'Environmental context', 'Table 1. Critical Criteria of Mahakam Delta Mangrove', 'Biodiversity issues', 'Institutional Context', 'The Movement of Indonesian New Capital', 'Aquaculture industry', 'Demography', 'Other issues', 'Strategy & theory of change', 'Lack of alternative sustainable livelihood in coastal area;', 'Environmental awareness;', 'Beneficiaries', 'Teachers (primary school teachers)', 'Village officials', 'Location', 'CAPACITY BUILDING PLAN', 'Cross-cutting approaches', 'Knowledge management', 'Capitalization', 'External communication in Indonesia', 'External communication in France', 'Sustainability, scaling-up and/or exit strategy', 'Sustainability and Exit Strategy (Scale Up) of the proj

In [1]:
import requests
from docx import Document

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte des sections d'un document Word """
    doc = Document(doc_path)
    sections = {}
    current_section = "Section Générique"

    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            if len(text.split()) < 10 and text[-1] not in ".!?":
                current_section = text
                sections[current_section] = []
            else:
                if current_section not in sections:
                    sections[current_section] = []
                sections[current_section].append(text)

    return {sec: " ".join(content) for sec, content in sections.items() if content}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse et compréhension du texte """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

# Charger les sections attendues du PU_P01_PP (document de référence)
pp_sections = extract_text_by_sections("../data/projet_1/PU_P01_PP.docx")

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    # Construire une requête avancée pour Mixtral
    prompt = f"""
    Nous devons vérifier si **les thématiques et contenus** du document de référence (PU_P01_PP) sont bien abordés dans {aap_path}.  
    Voici les **thématiques et contenus** attendus :
    {list(pp_sections.keys())}

    **Contenu du document analysé (AAP) :**
    {list(aap_sections.keys())}

    **Tâches :**
    1️⃣ Pour chaque thématique attendue, vérifie si elle est bien abordée dans l'AAP, même sous un autre titre.  
    2️⃣ Si une thématique est **totalement couverte**, marque-la ✅.  
    3️⃣ Si une thématique est **partiellement couverte**, marque-la ⚠️ et explique ce qui manque.  
    4️⃣ Si une thématique est totalement absente, marque-la ❌.  
    5️⃣ Donne une liste claire des thématiques **partiellement ou totalement absentes**.  

    **Réponse attendue :**
    - ✅ [Nom de la thématique] : Bien couverte.
    - ⚠️ [Nom de la thématique] : Partiellement couverte, éléments manquants : [détails].
    - ❌ [Nom de la thématique] : Absente.
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats
for doc, result in results.items():
    print(f"\n===== Vérification thématique pour {doc} =====")
    print(result)



===== Vérification thématique pour ../data/projet_1/PU_P01_AAP01.docx =====
1. ✅ Brief project description : The AAP includes a section for describing the projects by year and items, which can cover this theme.
2. ⚠️ Expected results, planned activities & required resources : The AAP has a section for expected concrete activity results, but it does not explicitly mention planned activities or required resources.
3. ❌ Activity 2.1. Pres-assessment study on land suitability : This specific activity is not mentioned in the AAP.
4. ⚠️ Address: Jl. Pelita 3 RT.12 Gg.7 Kel. Sambutan Samarinda : The AAP includes a section for website URLs, but it does not explicitly mention an address for the organization.
5. ✅ Restoring mangrove ecosystems, especially in the Mahakam Delta : This theme is mentioned in the AAP as one of the project's activities.
6. ✅ Name of the organization: Pokja Pesisir : The name of the organization is mentioned in the AAP.
7. ✅ Website: <http://pokjapesisir.id/> : The we

### pourcentage

In [2]:
import requests
from docx import Document

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte des sections d'un document Word """
    doc = Document(doc_path)
    sections = {}
    current_section = "Section Générique"

    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            # Détection des titres de section (moins de 10 mots, sans ponctuation forte)
            if len(text.split()) < 10 and text[-1] not in ".!?":
                current_section = text
                sections[current_section] = []
            else:
                sections.setdefault(current_section, []).append(text)

    return {sec: " ".join(content) for sec, content in sections.items() if content}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse et comparaison du texte """
    response = requests.post(
        OLLAMA_HOST,
        json={"model": "mixtral", "prompt": prompt, "stream": False}
    )
    return response.json().get("response", "Erreur: pas de réponse")

# Charger les sections du document de référence PU_P01_PP
pp_sections = extract_text_by_sections("../data/projet_1/PU_P01_PP.docx")

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    # Construire une requête optimisée pour Mixtral
    prompt = f"""
    Nous analysons si **les thématiques et contenus** attendus (PU_P01_PP) sont bien couverts dans {aap_path}.
    
    **📌 Thématiques attendues :**
    {list(pp_sections.keys())}

    **📄 Thématiques présentes dans l'AAP analysé :**
    {list(aap_sections.keys())}

    **🎯 Tâches :**
    - **Associer chaque thématique attendue** à son équivalent dans l'AAP (même sous un autre nom).
    - **Évaluer la couverture** :
      - ✅ **Bien couvert** → La thématique est abordée en détail.
      - ⚠️ **Partiellement couvert** → Présente mais incomplète (détailler ce qui manque).
      - ❌ **Absent** → Pas abordé du tout.
    - **Lister les sections manquantes** avec des précisions sur ce qui fait défaut.
    - **Calculer un score de conformité (%)** basé sur les thématiques couvertes.

    **📌 Format de réponse attendu :**
    - 📊 **Score : X%** (Nb de thématiques couvertes / Total thématiques attendues)
    - ✅ [Nom de la thématique] : Couvert en détail.
    - ⚠️ [Nom de la thématique] : Partiellement couvert → Manque [éléments spécifiques].
    - ❌ [Nom de la thématique] : Absent.

    **🔎 Exemple attendu :**
    ```
    📊 Score : 75%
    ✅ Brief project description : Présent avec une bonne description.
    ⚠️ Expected results : Mentionnés mais sans détails sur les ressources nécessaires.
    ❌ Monitoring & evaluation : Aucun élément sur le suivi des résultats.
    ```
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats formatés
for doc, result in results.items():
    print(f"\n===== 📊 Vérification thématique pour {doc} =====")
    print(result)



===== 📊 Vérification thématique pour ../data/projet_1/PU_P01_AAP01.docx =====
📊 Score : 25%

✅ Brief project description : Présent dans l'AAP sous "Section Générique" avec une brève description du projet.

❌ Thématiques et contenus attendus (hors titre) : Aucune correspondance directe trouvée dans l'AAP analysé. Les thématiques présentes dans l'AAP concernent principalement les activités planifiées, les résultats concrets attendus et le calendrier de l'activité, mais ne couvrent pas les autres sujets mentionnés dans la liste des thématiques attendues.

Sections manquantes :

* Restoring mangrove ecosystems, especially in the Mahakam Delta
* Name of the organization: Pokja Pesisir
* Website: <http://pokjapesisir.id/>
* Geographic and socio-economic context
* Environmental context
* Biodiversity issues
* Institutional Context
* The Movement of Indonesian New Capital
* Aquaculture industry
* Demography
* Other issues
* Strategy & theory of change
* Lack of alternative sustainable livelih

In [ ]:
import requests
from docx import Document
import os

OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"

def extract_text_by_sections(doc_path):
    """ Extrait le texte des sections d'un document Word en structurant le contenu """
    if not os.path.exists(doc_path):
        print(f"Erreur : fichier introuvable {doc_path}")
        return {}

    doc = Document(doc_path)
    sections = {}
    current_section = "Section Générique"

    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            if len(text.split()) < 10 and text[-1] not in ".!?":  
                current_section = text.lower().strip()
                sections[current_section] = []
            else:
                sections.setdefault(current_section, []).append(text)

    return {sec: " ".join(content) for sec, content in sections.items() if content}

def ask_mixtral(prompt):
    """ Envoie une requête à Mixtral pour analyse et comparaison du texte """
    try:
        response = requests.post(
            OLLAMA_HOST,
            json={"model": "mixtral", "prompt": prompt, "stream": False}
        )
        response.raise_for_status()
        return response.json().get("response", "Erreur: pas de réponse")
    except requests.RequestException as e:
        return f"Erreur lors de la requête : {e}"

# Charger les sections du document de référence PU_P01_PP
pp_path = "../data/projet_1/PU_P01_PP.docx"
pp_sections = extract_text_by_sections(pp_path)

# Charger et comparer les AAP
aap_paths = ["../data/projet_1/PU_P01_AAP01.docx", "../data/projet_1/PU_P01_AAP02.docx", "../data/projet_1/PU_P01_AAP03.docx"]
results = {}

for aap_path in aap_paths:
    aap_sections = extract_text_by_sections(aap_path)

    if not aap_sections:
        print(f"⚠️ Aucun contenu extrait de {aap_path}")
        continue

    # Construire une requête optimisée pour Mixtral
    prompt = f"""
    Nous analysons si les thématiques et contenus attendus (PU_P01_PP) sont bien couverts dans {os.path.basename(aap_path)}.

    📌 **Thématiques attendues :**
    {list(pp_sections.keys())}

    📄 **Thématiques présentes dans l'AAP analysé :**
    {list(aap_sections.keys())}

    🎯 **Tâches :**
    - Associer chaque thématique attendue à son équivalent dans l'AAP (même sous un autre nom).
    - Évaluer la couverture :
      - ✅ **Bien couvert** → Thématique détaillée.
      - ⚠️ **Partiellement couvert** → Présente mais incomplète (indiquer les manques).
      - ❌ **Absent** → Pas abordé du tout.
    - Identifier les sections manquantes et préciser ce qui fait défaut.
    - Calculer un **score de conformité (%)** basé sur les thématiques couvertes.

    📊 **Format de réponse attendu :**
    ```
    📊 Score : X%
    ✅ [Nom de la thématique] : Bien détaillée.
    ⚠️ [Nom de la thématique] : Partiellement couverte → Manque [éléments précis].
    ❌ [Nom de la thématique] : Absente.
    ```
    """

    mixtral_response = ask_mixtral(prompt)
    results[aap_path] = mixtral_response

# Affichage des résultats formatés
for doc, result in results.items():
    print(f"\n===== 📊 Vérification thématique pour {os.path.basename(doc)} =====")
    print(result)



===== 📊 Vérification thématique pour PU_P01_AAP01.docx =====
📊 Score : 15%

✅ brief project description: Bien détaillée dans la section "3. expected concrete activity results" de l'AAP analysé.

❌ '6.expected results, planned activities & required resources\t13': Absente. Aucune information concernant les résultats attendus, les activités prévues et les ressources requises n'a été trouvée dans l'AAP analysé.

⚠️ activity 2.1. pres-assessment study on land suitability\t17: Partiellement couverte. Bien que la section "4. activity schedule" mentionne une étude de faisabilité, il manque des informations spécifiques sur l'évaluation de la pertinence du terrain pour la restauration des écosystèmes de mangroves.

❌ address: jl. pelita 3 rt.12 gg.7 kel. sambutan samarinda: Absente. L'adresse de l'organisation n'est pas spécifiée dans l'AAP analysé, uniquement sur le document original fourni.

✅ restoring mangrove ecosystems, especially in the mahakam delta: Bien détaillée dans la section "3. 

## Exploring different AAP

The goal to reach is about to find beetwween all the AAP, the common struct or what are the needs in it

In [3]:
pip install PyPDF2


  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [5]:
pip install pandas


  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [11]:
import requests
from docx import Document
import os
from pathlib import Path
import json
import PyPDF2
import pandas as pd

def extract_text_from_pdf(file_path):
    """Extrait le texte d'un fichier PDF"""
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print(f"Erreur lors de la lecture du PDF {file_path}: {e}")
        return ""

def extract_text_from_excel(file_path):
    """Extrait le texte d'un fichier Excel"""
    try:
        df = pd.read_excel(file_path)
        text = "\n".join([str(col) for col in df.columns])
        for _, row in df.iterrows():
            text += "\n" + "\n".join([str(cell) for cell in row])
        return text
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier Excel {file_path}: {e}")
        return ""

def extract_text_by_sections(file_path):
    """Extrait le texte des sections d'un document en fonction de son format"""
    if not os.path.exists(file_path):
        print(f"Erreur : fichier introuvable {file_path}")
        return {}

    sections = {"contenu_general": []}
    
    try:
        if file_path.lower().endswith('.docx'):
            doc = Document(file_path)
            current_section = "contenu_general"
            
            for para in doc.paragraphs:
                text = para.text.strip()
                if text:
                    if len(text.split()) < 10 and text[-1] not in ".!?":
                        current_section = text.lower().strip()
                        sections.setdefault(current_section, [])
                    else:
                        sections[current_section].append(text)
                        
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)
            sections["contenu_general"].append(text)
            
        elif file_path.lower().endswith(('.xlsx', '.xls')):
            text = extract_text_from_excel(file_path)
            sections["contenu_general"].append(text)
            
        else:
            print(f"Format de fichier non supporté : {file_path}")
            return {}

        return {sec: " ".join(content) for sec, content in sections.items() if content}
        
    except Exception as e:
        print(f"Erreur lors de l'extraction du fichier {file_path}: {e}")
        return {}

def find_aap_files(root_dir):
    """Trouve tous les fichiers AAP dans l'arborescence"""
    aap_files = []
    root = Path(root_dir)
    
    for path in root.rglob("*"):
        if path.is_file() and "AAP" in path.name.upper():
            aap_files.append(str(path))
    
    return aap_files

def analyze_topics_with_mixtral(all_sections):
    """Analyse les sections communes et spécifiques avec Mixtral"""
    try:
        # Préparation des données pour l'analyse
        sections_count = {}
        for doc, sections in all_sections.items():
            for section in sections:
                sections_count[section] = sections_count.get(section, 0) + 1

        prompt = f"""
        Analyse des documents AAP (Appels À Projets) :
        
        Documents analysés : {len(all_sections)} fichiers
        
        Tâches :
        1. Identifier les sections communes qui apparaissent dans plusieurs documents
        2. Créer un template générique basé sur les sections les plus fréquentes
        3. Pour chaque document, identifier ses spécificités
        4. Calculer des statistiques sur la fréquence des sections
        
        Données :
        {json.dumps(all_sections, indent=2, ensure_ascii=False)}
        
        Statistiques préliminaires :
        {json.dumps(sections_count, indent=2, ensure_ascii=False)}
        
        Format de réponse souhaité :
        ```
        📋 Template Commun :
        - [Section] : Présente dans X% des documents
        - [Section] : Présente dans Y% des documents
        ...
        
        🔍 Spécificités par document :
        [Document 1]
        - Sections uniques : [liste]
        - Contenu particulier : [description]
        ...
        
        📊 Statistiques :
        - Nombre moyen de sections : X
        - Top 5 sections les plus fréquentes
        - Top 5 sections les plus rares
        ```
        
        Répondez en français et soyez précis dans l'analyse des sections communes et spécifiques.
        """
        # OLLAMA_HOST = "http://127.0.0.1:11435/api/generate"
        response = requests.post(
            "http://127.0.0.1:11434/api/generate",
            json={
                "model": "mixtral:latest",
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.7,
                    "top_p": 0.9,
                    "num_predict": 2048
                }
            },
            timeout=120
        )
        response.raise_for_status()
        return response.json().get("response", "Erreur: pas de réponse")
    except requests.RequestException as e:
        print(f"Erreur détaillée de la requête : {str(e)}")
        return f"Erreur lors de la requête Mixtral : {e}\nVérifiez qu'Ollama est bien lancé sur le port 11435"
    except Exception as e:
        print(f"Erreur détaillée : {str(e)}")
        return f"Erreur inattendue : {e}"

def main():
    # Configuration
    data_dir = "../data"
    print("🔍 Recherche des fichiers AAP...")
    aap_files = find_aap_files(data_dir)
    print(f"📁 Nombre de fichiers AAP trouvés : {len(aap_files)}")

    # Extraction des sections
    all_sections = {}
    for file_path in aap_files:
        print(f"\n📄 Analyse de {os.path.basename(file_path)}")
        sections = extract_text_by_sections(file_path)
        if sections:
            all_sections[os.path.basename(file_path)] = list(sections.keys())

    # Analyse avec Mixtral
    print("\n🤖 === Analyse des topics avec Mixtral ===")
    analysis = analyze_topics_with_mixtral(all_sections)
    print(analysis)

    # Sauvegarde des résultats
    results = {
        "nombre_fichiers": len(aap_files),
        "fichiers_analyses": list(all_sections.keys()),
        "sections_par_fichier": all_sections,
        "analyse_mixtral": analysis
    }
    
    output_file = "resultats_analyse_aap.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
        print(f"\n💾 Résultats sauvegardés dans {output_file}")

if __name__ == "__main__":
    main()

🔍 Recherche des fichiers AAP...
📁 Nombre de fichiers AAP trouvés : 33

📄 Analyse de Guide Pratique Réponse AAP (Groupe SOS).pdf

📄 Analyse de ESF_P01_AAP01.docx

📄 Analyse de ESF_P01_AAP02.docx

📄 Analyse de ESF_P01_AAP03.docx

📄 Analyse de ESF_P01_AAP04.docx

📄 Analyse de PU_P01_AAP01.docx

📄 Analyse de PU_P01_AAP02.docx

📄 Analyse de PU_P01_AAP03.docx

📄 Analyse de Pulse_P01_AAP01.docx

📄 Analyse de Pulse_P02_AAP01.rtf
Format de fichier non supporté : ..\data\Pulse\Projet02\Pulse_P02_AAP01.rtf

📄 Analyse de Pulse_P02_AAP01E
Format de fichier non supporté : ..\data\Pulse\Projet02\Pulse_P02_AAP01E

📄 Analyse de PS_P03_AAP01.docx

📄 Analyse de dossier_de_candidature_aap_projets_innov_entre_2024.docx

📄 Analyse de Fiche candidature AAP jeunes pousses 2024-2026 (2).pdf

📄 Analyse de Note cadrage AAP.docx

📄 Analyse de Objectifs et réalisations AAP jeunes pousses 2024-2026.xlsx
Erreur lors de la lecture du fichier Excel ..\data\Pulse\Projet04\Pulse_P04_APP1\Objectifs et réalisations AAP j

### with a specific template ( extract from the last observation )



In [5]:
import requests
import json
from pathlib import Path
import os
from typing import Dict, List, Any
from docx import Document
import PyPDF2
import pandas as pd
from collections import Counter
import openpyxl
class AAPAnalyzer:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        
    def extract_text_by_sections(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte par sections d'un fichier"""
        try:
            if file_path.lower().endswith('.docx'):
                return self._extract_from_docx(file_path)
            elif file_path.lower().endswith('.pdf'):
                return self._extract_from_pdf(file_path)
            elif file_path.lower().endswith(('.xlsx', '.xls')):
                return self._extract_from_excel(file_path)
            else:
                print(f"⚠️ Format non supporté: {file_path}")
                return {}
        except Exception as e:
            print(f"❌ Erreur extraction {file_path}: {e}")
            return {}

    def _extract_from_docx(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier Word"""
        sections = {}
        current_section = "Introduction"
        current_content = []
        
        try:
            doc = Document(file_path)
            for para in doc.paragraphs:
                text = para.text.strip()
                if not text:
                    continue
                
                # Détection d'une nouvelle section
                if text.startswith('###') or (para.style and "Heading" in para.style.name):
                    if current_content:
                        sections[current_section] = "\n".join(current_content)
                    current_section = text.strip('#').strip()
                    current_content = []
                else:
                    current_content.append(text)
            
            # Ajouter la dernière section
            if current_content:
                sections[current_section] = "\n".join(current_content)
                
            return sections
            
        except Exception as e:
            print(f"❌ Erreur Word {file_path}: {e}")
            return {}

    def _extract_from_pdf(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier PDF"""
        sections = {}
        current_section = "Introduction"
        current_content = []
        
        try:
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    content = page.extract_text()
                    lines = content.split('\n')
                    
                    for line in lines:
                        line = line.strip()
                        if not line:
                            continue
                        
                        # Détection d'une nouvelle section
                        if line.startswith('###') or line.isupper():
                            if current_content:
                                sections[current_section] = "\n".join(current_content)
                            current_section = line.strip('#').strip()
                            current_content = []
                        else:
                            current_content.append(line)
            
            # Ajouter la dernière section
            if current_content:
                sections[current_section] = "\n".join(current_content)
                
            return sections
            
        except Exception as e:
            print(f"❌ Erreur PDF {file_path}: {e}")
            return {}

    def _extract_from_excel(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier Excel"""
        sections = {}
        
        try:
            df = pd.read_excel(file_path)
            for col in df.columns:
                if isinstance(col, str) and col.strip():
                    content = df[col].dropna().astype(str).tolist()
                    if content:
                        sections[col] = "\n".join(content)
            return sections
            
        except Exception as e:
            print(f"❌ Erreur Excel {file_path}: {e}")
            return {}

    def analyze_topics(self, all_sections: Dict[str, Dict[str, str]]) -> str:
        """Analyse principale des documents"""
        try:
            print("🔍 Analyse document par document...")
            all_analyses = {}
            
            for doc_name, sections in all_sections.items():
                print(f"\n📄 Analyse de {doc_name}")
                doc_analysis = self.analyze_document(sections)
                all_analyses[doc_name] = doc_analysis
            
            print("\n🔄 Comparaison des documents...")
            comparison = self.compare_documents_themes(all_analyses)
            
            print("\n📊 Génération du rapport...")
            report = self.generate_comparative_report(comparison)
            
            # Sauvegarde des résultats détaillés
            self.save_detailed_results(all_analyses, comparison)
            
            return report
            
        except Exception as e:
            print(f"❌ Erreur analyse: {e}")
            return "Erreur lors de l'analyse"
    
    def analyze_content_with_mixtral(self, text: str, section_name: str) -> dict:
        """Analyse sémantique d'une section spécifique"""
        try:
            prompt = f"""Tu es un expert en analyse de documents. Analyse cette section et renvoie UNIQUEMENT un JSON valide sans aucun autre texte.

    SECTION À ANALYSER : {section_name}
    CONTENU : {text}

    Le JSON DOIT suivre EXACTEMENT ce format :
    {{
        "themes": [
            {{
                "nom": "nom de la thématique",
                "concepts_cles": ["liste des concepts"],
                "points_importants": ["liste des points"],
                "specificites": ["éléments spécifiques"]
            }}
        ],
        "resume": "résumé court de la section",
        "mots_cles": ["liste des mots-clés importants"]
    }}"""

            response = requests.post(
                "http://localhost:11434/api/generate",
                json={
                    "model": "mixtral:latest",
                    "prompt": prompt,
                    "temperature": 0.7,
                    "stream": False,
                    "format": "json"  # Forcer le format JSON
                },
                timeout=180
            )
            
            if response.status_code == 200:
                try:
                    # Récupérer la réponse
                    response_data = response.json()
                    
                    # Extraire le JSON de la réponse
                    if isinstance(response_data, dict) and "response" in response_data:
                        try:
                            # Tenter de parser le JSON dans la réponse
                            json_str = response_data["response"]
                            # Nettoyer la chaîne JSON si nécessaire
                            json_str = json_str.strip()
                            if json_str.startswith("```json"):
                                json_str = json_str[7:]
                            if json_str.endswith("```"):
                                json_str = json_str[:-3]
                            
                            parsed_data = json.loads(json_str)
                            
                            # Vérifier la structure
                            if not isinstance(parsed_data, dict):
                                raise ValueError("La réponse n'est pas un objet JSON")
                            
                            # S'assurer que les champs requis sont présents
                            if "themes" not in parsed_data:
                                parsed_data["themes"] = []
                            if "resume" not in parsed_data:
                                parsed_data["resume"] = ""
                            if "mots_cles" not in parsed_data:
                                parsed_data["mots_cles"] = []
                                
                            return parsed_data
                            
                        except json.JSONDecodeError as e:
                            print(f"⚠️ Erreur format JSON pour la section '{section_name}': {str(e)}")
                            return {"themes": [], "resume": "", "mots_cles": []}
                            
                except Exception as e:
                    print(f"⚠️ Erreur parsing réponse pour la section '{section_name}': {str(e)}")
                    return {"themes": [], "resume": "", "mots_cles": []}
            else:
                print(f"⚠️ Erreur Mixtral ({response.status_code}) pour la section '{section_name}'")
                return {"themes": [], "resume": "", "mots_cles": []}
                
        except Exception as e:
            print(f"❌ Erreur analyse Mixtral pour la section '{section_name}': {str(e)}")
            return {"themes": [], "resume": "", "mots_cles": []}
    
    def analyze_document(self, sections: Dict[str, str]) -> Dict[str, Any]:
        """Analyse un document section par section"""
        document_analysis = {
            "sections_analysis": {},
            "global_themes": [],  # Liste au lieu de set
            "all_keywords": []    # Liste au lieu de set
        }
        
        for section_name, content in sections.items():
            print(f"  Analyse de la section : {section_name}")
            section_analysis = self.analyze_content_with_mixtral(content, section_name)
            
            document_analysis["sections_analysis"][section_name] = section_analysis
            
            # Collecter les thèmes et mots-clés
            for theme in section_analysis.get("themes", []):
                theme_nom = theme.get("nom", "")
                if theme_nom and theme_nom not in document_analysis["global_themes"]:
                    document_analysis["global_themes"].append(theme_nom)
                
                for concept in theme.get("concepts_cles", []):
                    if concept and concept not in document_analysis["all_keywords"]:
                        document_analysis["all_keywords"].append(concept)
        
        return document_analysis

    def compare_documents_themes(self, all_analyses: Dict[str, Dict]) -> Dict[str, Any]:
        """Compare les thématiques entre les documents"""
        comparison = {
            "themes_communs": {},
            "themes_uniques": {},
            "mots_cles_frequence": Counter(),
            "sections_similaires": {},
            "statistiques": {
                "nombre_documents": len(all_analyses),
                "themes_par_document": {},
                "sections_par_document": {}
            }
        }
        
        # Analyser chaque document
        for doc_name, analysis in all_analyses.items():
            themes = analysis["global_themes"]
            comparison["statistiques"]["themes_par_document"][doc_name] = len(themes)
            comparison["statistiques"]["sections_par_document"][doc_name] = len(analysis["sections_analysis"])
            
            # Compter la fréquence des thèmes
            for theme in themes:
                if theme not in comparison["themes_communs"]:
                    comparison["themes_communs"][theme] = 1
                else:
                    comparison["themes_communs"][theme] += 1
            
            # Collecter les mots-clés
            comparison["mots_cles_frequence"].update(analysis["all_keywords"])
        
        # Identifier les thèmes uniques et communs
        total_docs = len(all_analyses)
        for theme, count in comparison["themes_communs"].items():
            if count == 1:
                comparison["themes_uniques"][theme] = count
            comparison["themes_communs"][theme] = (count / total_docs) * 100
        
        return comparison

    def generate_comparative_report(self, comparison: Dict[str, Any]) -> str:
        """Génère un rapport comparatif détaillé"""
        report = []
        report.append("📊 ANALYSE COMPARATIVE DES DOCUMENTS")
        report.append(f"Nombre total de documents analysés : {comparison['statistiques']['nombre_documents']}\n")
        
        # Thèmes communs
        report.append("🔄 THÈMES COMMUNS")
        for theme, percentage in sorted(comparison["themes_communs"].items(), key=lambda x: x[1], reverse=True):
            if percentage > 100/comparison['statistiques']['nombre_documents']:
                report.append(f"• {theme}: {percentage:.1f}% des documents")
        
        # Thèmes uniques
        report.append("\n💫 THÈMES UNIQUES")
        for theme in comparison["themes_uniques"]:
            report.append(f"• {theme}")
        
        # Mots-clés les plus fréquents
        report.append("\n🔤 MOTS-CLÉS LES PLUS FRÉQUENTS")
        for mot, count in comparison["mots_cles_frequence"].most_common(10):
            report.append(f"• {mot}: {count} occurrences")
        
        # Statistiques par document
        report.append("\n📈 STATISTIQUES PAR DOCUMENT")
        for doc, nb_themes in comparison["statistiques"]["themes_par_document"].items():
            nb_sections = comparison["statistiques"]["sections_par_document"][doc]
            report.append(f"• {doc}:")
            report.append(f"  - Nombre de thèmes : {nb_themes}")
            report.append(f"  - Nombre de sections : {nb_sections}")
        
        return "\n".join(report)

    def analyze_topics(self, all_sections: Dict[str, Dict[str, str]]) -> str:
        """Analyse principale des documents"""
        try:
            print("🔍 Analyse document par document...")
            all_analyses = {}
            
            for doc_name, sections in all_sections.items():
                print(f"\n📄 Analyse de {doc_name}")
                doc_analysis = self.analyze_document(sections)
                all_analyses[doc_name] = doc_analysis
            
            print("\n🔄 Comparaison des documents...")
            comparison = self.compare_documents_themes(all_analyses)
            
            print("\n📊 Génération du rapport...")
            report = self.generate_comparative_report(comparison)
            
            # Sauvegarde des résultats détaillés
            self.save_detailed_results(all_analyses, comparison)
            
            return report
            
        except Exception as e:
            print(f"❌ Erreur analyse: {e}")
            return "Erreur lors de l'analyse"

    def calculate_statistics(self, analysis: Dict) -> Dict:
            """Calcule des statistiques détaillées sur l'analyse"""
            stats = {
                "nombre_total_documents": len(analysis.get("themes_principaux", [])),
                "themes": {
                    "count": len(analysis.get("themes_principaux", [])),
                    "distribution": {},
                    "mots_cles_frequents": Counter(),
                    "correlations": {}
                },
                "similarites": {
                    "count_patterns": len(analysis.get("similarites", {}).get("patterns_communs", [])),
                    "count_differences": len(analysis.get("similarites", {}).get("differences_notables", []))
                }
            }

            # Analyse des thèmes
            for theme in analysis.get("themes_principaux", []):
                nom = theme.get("nom", "")
                freq = float(theme.get("frequence", "0").strip("%"))/100
                stats["themes"]["distribution"][nom] = freq
                
                # Mots-clés fréquents
                for mot in theme.get("mots_cles", []):
                    stats["themes"]["mots_cles_frequents"][mot] += 1

            # Calcul des corrélations entre thèmes
            themes = list(stats["themes"]["distribution"].keys())
            for i, theme1 in enumerate(themes):
                for theme2 in themes[i+1:]:
                    correlation = self._calculate_theme_correlation(
                        analysis, theme1, theme2
                    )
                    stats["themes"]["correlations"][f"{theme1}-{theme2}"] = correlation

            return stats

    def analyze_document(self, sections: Dict[str, str]) -> Dict[str, Any]:
            """Analyse un document section par section"""
            document_analysis = {
                "sections_analysis": {},
                "global_themes": set(),
                "all_keywords": set()
            }
            
            for section_name, content in sections.items():
                print(f"  Analyse de la section : {section_name}")
                section_analysis = self.analyze_content_with_mixtral(content, section_name)
                
                document_analysis["sections_analysis"][section_name] = section_analysis
                
                # Collecter les thèmes et mots-clés
                for theme in section_analysis.get("themes", []):
                    document_analysis["global_themes"].add(theme["nom"])
                    document_analysis["all_keywords"].update(theme.get("concepts_cles", []))
            
            return document_analysis

    def compare_documents_themes(self, all_analyses: Dict[str, Dict]) -> Dict[str, Any]:
            """Compare les thématiques entre les documents"""
            comparison = {
                "themes_communs": {},
                "themes_uniques": {},
                "mots_cles_frequence": Counter(),
                "sections_similaires": {},
                "statistiques": {
                    "nombre_documents": len(all_analyses),
                    "themes_par_document": {},
                    "sections_par_document": {}
                }
            }
            
            # Analyser chaque document
            for doc_name, analysis in all_analyses.items():
                themes = analysis["global_themes"]
                comparison["statistiques"]["themes_par_document"][doc_name] = len(themes)
                comparison["statistiques"]["sections_par_document"][doc_name] = len(analysis["sections_analysis"])
                
                # Compter la fréquence des thèmes
                for theme in themes:
                    if theme not in comparison["themes_communs"]:
                        comparison["themes_communs"][theme] = 1
                    else:
                        comparison["themes_communs"][theme] += 1
                
                # Collecter les mots-clés
                comparison["mots_cles_frequence"].update(analysis["all_keywords"])
            
            # Identifier les thèmes uniques et communs
            total_docs = len(all_analyses)
            for theme, count in list(comparison["themes_communs"].items()):
                if count == 1:
                    comparison["themes_uniques"][theme] = count
                    del comparison["themes_communs"][theme]
                else:
                    comparison["themes_communs"][theme] = (count / total_docs) * 100
            
            return comparison

    def generate_comparative_report(self, comparison: Dict[str, Any]) -> str:
            """Génère un rapport comparatif détaillé"""
            report = []
            report.append("📊 ANALYSE COMPARATIVE DES DOCUMENTS")
            report.append(f"Nombre total de documents analysés : {comparison['statistiques']['nombre_documents']}\n")
            
            # Thèmes communs
            report.append("🔄 THÈMES COMMUNS")
            for theme, percentage in sorted(comparison["themes_communs"].items(), key=lambda x: x[1], reverse=True):
                report.append(f"• {theme}: {percentage:.1f}% des documents")
            
            # Thèmes uniques
            report.append("\n💫 THÈMES UNIQUES")
            for theme in comparison["themes_uniques"]:
                report.append(f"• {theme}")
            
            # Mots-clés les plus fréquents
            report.append("\n🔤 MOTS-CLÉS LES PLUS FRÉQUENTS")
            for mot, count in comparison["mots_cles_frequence"].most_common(10):
                report.append(f"• {mot}: {count} occurrences")
            
            # Statistiques par document
            report.append("\n📈 STATISTIQUES PAR DOCUMENT")
            for doc, nb_themes in comparison["statistiques"]["themes_par_document"].items():
                nb_sections = comparison["statistiques"]["sections_par_document"][doc]
                report.append(f"• {doc}:")
                report.append(f"  - Nombre de thèmes : {nb_themes}")
                report.append(f"  - Nombre de sections : {nb_sections}")
            
            return "\n".join(report)

    def analyze_topics(self, all_sections: Dict[str, Dict[str, str]]) -> str:
            """Analyse principale des documents"""
            try:
                print("🔍 Analyse document par document...")
                all_analyses = {}
                
                for doc_name, sections in all_sections.items():
                    print(f"\n📄 Analyse de {doc_name}")
                    doc_analysis = self.analyze_document(sections)
                    all_analyses[doc_name] = doc_analysis
                
                print("\n🔄 Comparaison des documents...")
                comparison = self.compare_documents_themes(all_analyses)
                
                print("\n📊 Génération du rapport...")
                report = self.generate_comparative_report(comparison)
                
                # Sauvegarde des résultats détaillés
                self.save_detailed_results(all_analyses, comparison)
                
                return report
                
            except Exception as e:
                print(f"❌ Erreur analyse: {e}")
                return "Erreur lors de l'analyse"

    def save_detailed_results(self, analyses: Dict, comparison: Dict):
        """Sauvegarde les résultats détaillés"""
        try:
            # Conversion des Counters en dictionnaires
            if "mots_cles_frequence" in comparison:
                comparison["mots_cles_frequence"] = dict(comparison["mots_cles_frequence"])
            
            # Préparation des résultats
            results = {
                "analyses_par_document": {
                    doc_name: {
                        "sections_analysis": doc_analysis["sections_analysis"],
                        "global_themes": doc_analysis["global_themes"],
                        "all_keywords": doc_analysis["all_keywords"]
                    }
                    for doc_name, doc_analysis in analyses.items()
                },
                "comparaison": comparison
            }
            
            with open("analyse_thematique_detaillee.json", "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
                
        except Exception as e:
            print(f"❌ Erreur sauvegarde résultats: {e}")
def main():
    analyzer = AAPAnalyzer("../data")
    
    print("🔍 Analyse des AAP en cours...")
    aap_files = [p for p in Path("../data").rglob("*") 
                 if p.is_file() and "AAP" in p.name.upper()]
    
    if not aap_files:
        print("❌ Aucun fichier AAP trouvé!")
        return
        
    print(f"📁 {len(aap_files)} fichiers AAP trouvés")
    
    all_sections = {}
    for file_path in aap_files:
        print(f"\n📄 Analyse de {file_path.name}")
        sections = analyzer.extract_text_by_sections(str(file_path))
        if sections:
            all_sections[file_path.name] = sections
    
    if not all_sections:
        print("❌ Aucune section extraite des fichiers!")
        return
    
    print("\n🤖 Analyse thématique en cours...")
    analysis = analyzer.analyze_topics(all_sections)
    
    print("\n=== Rapport d'Analyse ===")
    print(analysis)

In [6]:
if __name__ == "__main__":
    main()

🔍 Analyse des AAP en cours...
📁 33 fichiers AAP trouvés

📄 Analyse de Guide Pratique Réponse AAP (Groupe SOS).pdf

📄 Analyse de ESF_P01_AAP01.docx

📄 Analyse de ESF_P01_AAP02.docx

📄 Analyse de ESF_P01_AAP03.docx

📄 Analyse de ESF_P01_AAP04.docx

📄 Analyse de PU_P01_AAP01.docx

📄 Analyse de PU_P01_AAP02.docx

📄 Analyse de PU_P01_AAP03.docx

📄 Analyse de Pulse_P01_AAP01.docx

📄 Analyse de Pulse_P02_AAP01.rtf
⚠️ Format non supporté: ..\data\Pulse\Projet02\Pulse_P02_AAP01.rtf

📄 Analyse de Pulse_P02_AAP01E
⚠️ Format non supporté: ..\data\Pulse\Projet02\Pulse_P02_AAP01E

📄 Analyse de PS_P03_AAP01.docx

📄 Analyse de dossier_de_candidature_aap_projets_innov_entre_2024.docx

📄 Analyse de Fiche candidature AAP jeunes pousses 2024-2026 (2).pdf

📄 Analyse de Note cadrage AAP.docx

📄 Analyse de Objectifs et réalisations AAP jeunes pousses 2024-2026.xlsx

📄 Analyse de Réponse_dossier_de_candidature_aap_projets_innov_entre_2024.pdf

📄 Analyse de Réponse_Fiche candidature AAP jeunes pousses 2024-20

### test further: more generalist approach

In [17]:
import requests
import json
from pathlib import Path
import os
from typing import Dict, List, Any
from docx import Document
import PyPDF2
import pandas as pd
from collections import Counter
import openpyxl
class AAPAnalyzer:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        
    def extract_text_by_sections(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte par sections d'un fichier"""
        try:
            if file_path.lower().endswith('.docx'):
                return self._extract_from_docx(file_path)
            elif file_path.lower().endswith('.pdf'):
                return self._extract_from_pdf(file_path)
            elif file_path.lower().endswith(('.xlsx', '.xls')):
                return self._extract_from_excel(file_path)
            else:
                print(f"⚠️ Format non supporté: {file_path}")
                return {}
        except Exception as e:
            print(f"❌ Erreur extraction {file_path}: {e}")
            return {}

    def _extract_from_docx(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier Word"""
        sections = {}
        current_section = "Introduction"
        current_content = []
        
        try:
            doc = Document(file_path)
            for para in doc.paragraphs:
                text = para.text.strip()
                if not text:
                    continue
                
                # Détection d'une nouvelle section
                if text.startswith('###') or (para.style and "Heading" in para.style.name):
                    if current_content:
                        sections[current_section] = "\n".join(current_content)
                    current_section = text.strip('#').strip()
                    current_content = []
                else:
                    current_content.append(text)
            
            # Ajouter la dernière section
            if current_content:
                sections[current_section] = "\n".join(current_content)
                
            return sections
            
        except Exception as e:
            print(f"❌ Erreur Word {file_path}: {e}")
            return {}

    def _extract_from_pdf(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier PDF"""
        sections = {}
        current_section = "Introduction"
        current_content = []
        
        try:
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    content = page.extract_text()
                    lines = content.split('\n')
                    
                    for line in lines:
                        line = line.strip()
                        if not line:
                            continue
                        
                        # Détection d'une nouvelle section
                        if line.startswith('###') or line.isupper():
                            if current_content:
                                sections[current_section] = "\n".join(current_content)
                            current_section = line.strip('#').strip()
                            current_content = []
                        else:
                            current_content.append(line)
            
            # Ajouter la dernière section
            if current_content:
                sections[current_section] = "\n".join(current_content)
                
            return sections
            
        except Exception as e:
            print(f"❌ Erreur PDF {file_path}: {e}")
            return {}

    def _extract_from_excel(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier Excel"""
        sections = {}
        
        try:
            df = pd.read_excel(file_path)
            for col in df.columns:
                if isinstance(col, str) and col.strip():
                    content = df[col].dropna().astype(str).tolist()
                    if content:
                        sections[col] = "\n".join(content)
            return sections
            
        except Exception as e:
            print(f"❌ Erreur Excel {file_path}: {e}")
            return {}

    def analyze_topics(self, all_sections: Dict[str, Dict[str, str]]) -> str:
        """Analyse principale des documents"""
        try:
            print("🔍 Analyse document par document...")
            all_analyses = {}
            
            for doc_name, sections in all_sections.items():
                print(f"\n📄 Analyse de {doc_name}")
                doc_analysis = self.analyze_document(sections)
                all_analyses[doc_name] = doc_analysis
            
            print("\n🔄 Comparaison des documents...")
            comparison = self.compare_documents_themes(all_analyses)
            
            print("\n📊 Génération du rapport...")
            report = self.generate_comparative_report(comparison)
            
            # Sauvegarde des résultats détaillés
            self.save_detailed_results(all_analyses, comparison)
            
            return report
            
        except Exception as e:
            print(f"❌ Erreur analyse: {e}")
            return "Erreur lors de l'analyse"
    
    def analyze_content_with_mixtral(self, text: str, section_name: str) -> dict:
        """Analyse sémantique d'une section spécifique"""
        try:
            prompt = f"""Tu es un expert en analyse thématique de documents. Analyse cette section et extrait les thèmes principaux.
    Pour chaque thème identifié, donne un score de confiance entre 0 et 1.

    SECTION : {section_name}
    CONTENU : {text}

    Renvoie UNIQUEMENT un JSON valide avec ce format exact :
    {{
        "themes": [
            {{
                "nom": "nom du thème",
                "score": 0.95,
                "sous_themes": ["liste des sous-thèmes"],
                "mots_cles": ["mots-clés importants"],
                "description": "courte description du thème"
            }}
        ],
        "resume": "résumé de la section"
    }}"""

            response = requests.post(
                "http://localhost:11434/api/generate",
                json={
                    "model": "mixtral:latest",
                    "prompt": prompt,
                    "temperature": 0.7,
                    "stream": False
                },
                timeout=180
            )
            
            if response.status_code == 200:
                try:
                    response_data = response.json()
                    if isinstance(response_data, dict) and "response" in response_data:
                        json_str = response_data["response"].strip()
                        if json_str.startswith("```json"):
                            json_str = json_str[7:]
                        if json_str.endswith("```"):
                            json_str = json_str[:-3]
                        
                        return json.loads(json_str)
                except:
                    print(f"⚠️ Erreur parsing JSON pour {section_name}")
                    return {"themes": [], "resume": ""}
            return {"themes": [], "resume": ""}
                
        except Exception as e:
            print(f"❌ Erreur Mixtral: {e}")
            return {"themes": [], "resume": ""}

    def analyze_document_themes(self, sections: Dict[str, str]) -> Dict[str, Any]:
        """Analyse les thèmes d'un document"""
        themes_by_section = {}
        global_themes = {}
        
        for section_name, content in sections.items():
            print(f"  Analyse de la section : {section_name}")
            analysis = self.analyze_content_with_mixtral(content, section_name)
            
            themes_by_section[section_name] = analysis
            
            # Agréger les thèmes globaux
            for theme in analysis.get("themes", []):
                theme_name = theme["nom"]
                if theme_name not in global_themes:
                    global_themes[theme_name] = {
                        "score_moyen": theme["score"],
                        "occurrences": 1,
                        "sections": [section_name],
                        "sous_themes": set(theme.get("sous_themes", [])),
                        "mots_cles": set(theme.get("mots_cles", [])),
                        "descriptions": [theme.get("description", "")]
                    }
                else:
                    global_themes[theme_name]["score_moyen"] += theme["score"]
                    global_themes[theme_name]["occurrences"] += 1
                    global_themes[theme_name]["sections"].append(section_name)
                    global_themes[theme_name]["sous_themes"].update(theme.get("sous_themes", []))
                    global_themes[theme_name]["mots_cles"].update(theme.get("mots_cles", []))
                    global_themes[theme_name]["descriptions"].append(theme.get("description", ""))
        
        # Calculer les scores moyens
        for theme in global_themes.values():
            theme["score_moyen"] /= theme["occurrences"]
            # Convertir les sets en listes pour la sérialisation JSON
            theme["sous_themes"] = list(theme["sous_themes"])
            theme["mots_cles"] = list(theme["mots_cles"])
        
        return {
            "themes_by_section": themes_by_section,
            "global_themes": global_themes
        }

    def compare_themes_across_documents(self, all_analyses: Dict[str, Dict]) -> Dict[str, Any]:
        """Compare les thèmes entre les documents"""
        theme_occurrences = {}  # Thème -> {documents: [], score_total: 0}
        theme_correlations = {}  # (theme1, theme2) -> score de corrélation
        
        # Collecter les occurrences des thèmes
        for doc_name, analysis in all_analyses.items():
            global_themes = analysis["global_themes"]
            for theme_name, theme_data in global_themes.items():
                if theme_name not in theme_occurrences:
                    theme_occurrences[theme_name] = {
                        "documents": [],
                        "score_total": 0,
                        "mots_cles": set(),
                        "sous_themes": set()
                    }
                
                theme_occurrences[theme_name]["documents"].append(doc_name)
                theme_occurrences[theme_name]["score_total"] += theme_data["score_moyen"]
                theme_occurrences[theme_name]["mots_cles"].update(theme_data["mots_cles"])
                theme_occurrences[theme_name]["sous_themes"].update(theme_data["sous_themes"])
        
        # Calculer les corrélations entre thèmes
        themes = list(theme_occurrences.keys())
        for i, theme1 in enumerate(themes):
            for theme2 in themes[i+1:]:
                # Calculer la similarité basée sur les documents partagés et les mots-clés
                docs1 = set(theme_occurrences[theme1]["documents"])
                docs2 = set(theme_occurrences[theme2]["documents"])
                mots1 = theme_occurrences[theme1]["mots_cles"]
                mots2 = theme_occurrences[theme2]["mots_cles"]
                
                # Coefficient de Jaccard pour les documents et mots-clés
                doc_sim = len(docs1 & docs2) / len(docs1 | docs2) if docs1 | docs2 else 0
                mot_sim = len(mots1 & mots2) / len(mots1 | mots2) if mots1 | mots2 else 0
                
                correlation = (doc_sim + mot_sim) / 2
                if correlation > 0.3:  # Seuil minimal de corrélation
                    theme_correlations[f"{theme1}-{theme2}"] = correlation
        
        # Convertir les sets en listes pour JSON
        for theme_data in theme_occurrences.values():
            theme_data["mots_cles"] = list(theme_data["mots_cles"])
            theme_data["sous_themes"] = list(theme_data["sous_themes"])
        
        return {
            "theme_occurrences": theme_occurrences,
            "theme_correlations": theme_correlations
        }

    def generate_theme_report(self, comparison: Dict[str, Any]) -> str:
        """Génère un rapport détaillé des thèmes"""
        report = []
        report.append("📊 ANALYSE THÉMATIQUE DES DOCUMENTS")
        
        # Thèmes les plus fréquents
        theme_occurrences = comparison["theme_occurrences"]
        report.append("\n🔄 THÈMES PRINCIPAUX")
        sorted_themes = sorted(
            theme_occurrences.items(),
            key=lambda x: len(x[1]["documents"]),
            reverse=True
        )
        
        for theme_name, theme_data in sorted_themes:
            nb_docs = len(theme_data["documents"])
            score_moyen = theme_data["score_total"] / nb_docs
            report.append(f"\n• {theme_name}")
            report.append(f"  - Présent dans {nb_docs} documents")
            report.append(f"  - Score moyen : {score_moyen:.2f}")
            report.append(f"  - Sous-thèmes : {', '.join(theme_data['sous_themes'][:5])}")
            report.append(f"  - Mots-clés principaux : {', '.join(theme_data['mots_cles'][:5])}")
        
        # Corrélations entre thèmes
        report.append("\n🔗 THÈMES CORRÉLÉS")
        sorted_correlations = sorted(
            comparison["theme_correlations"].items(),
            key=lambda x: x[1],
            reverse=True
        )
        
        for theme_pair, correlation in sorted_correlations[:10]:
            theme1, theme2 = theme_pair.split("-")
            report.append(f"• {theme1} ⟷ {theme2}: {correlation:.2f}")
        
        return "\n".join(report)

    def analyze_topics(self, all_sections: Dict[str, Dict[str, str]]) -> str:
        """Analyse principale des documents"""
        try:
            print("🔍 Analyse document par document...")
            all_analyses = {}
            
            for doc_name, sections in all_sections.items():
                print(f"\n📄 Analyse de {doc_name}")
                doc_analysis = self.analyze_document(sections)
                all_analyses[doc_name] = doc_analysis
            
            print("\n🔄 Comparaison des documents...")
            comparison = self.compare_documents_themes(all_analyses)
            
            print("\n📊 Génération du rapport...")
            report = self.generate_comparative_report(comparison)
            
            # Sauvegarde des résultats détaillés
            self.save_detailed_results(all_analyses, comparison)
            
            return report
            
        except Exception as e:
            print(f"❌ Erreur analyse: {e}")
            return "Erreur lors de l'analyse"

    def calculate_statistics(self, analysis: Dict) -> Dict:
            """Calcule des statistiques détaillées sur l'analyse"""
            stats = {
                "nombre_total_documents": len(analysis.get("themes_principaux", [])),
                "themes": {
                    "count": len(analysis.get("themes_principaux", [])),
                    "distribution": {},
                    "mots_cles_frequents": Counter(),
                    "correlations": {}
                },
                "similarites": {
                    "count_patterns": len(analysis.get("similarites", {}).get("patterns_communs", [])),
                    "count_differences": len(analysis.get("similarites", {}).get("differences_notables", []))
                }
            }

            # Analyse des thèmes
            for theme in analysis.get("themes_principaux", []):
                nom = theme.get("nom", "")
                freq = float(theme.get("frequence", "0").strip("%"))/100
                stats["themes"]["distribution"][nom] = freq
                
                # Mots-clés fréquents
                for mot in theme.get("mots_cles", []):
                    stats["themes"]["mots_cles_frequents"][mot] += 1

            # Calcul des corrélations entre thèmes
            themes = list(stats["themes"]["distribution"].keys())
            for i, theme1 in enumerate(themes):
                for theme2 in themes[i+1:]:
                    correlation = self._calculate_theme_correlation(
                        analysis, theme1, theme2
                    )
                    stats["themes"]["correlations"][f"{theme1}-{theme2}"] = correlation

            return stats

    def analyze_document(self, sections: Dict[str, str]) -> Dict[str, Any]:
            """Analyse un document section par section"""
            document_analysis = {
                "sections_analysis": {},
                "global_themes": set(),
                "all_keywords": set()
            }
            
            for section_name, content in sections.items():
                print(f"  Analyse de la section : {section_name}")
                section_analysis = self.analyze_content_with_mixtral(content, section_name)
                
                document_analysis["sections_analysis"][section_name] = section_analysis
                
                # Collecter les thèmes et mots-clés
                for theme in section_analysis.get("themes", []):
                    document_analysis["global_themes"].add(theme["nom"])
                    document_analysis["all_keywords"].update(theme.get("concepts_cles", []))
            
            return document_analysis

    def compare_documents_themes(self, all_analyses: Dict[str, Dict]) -> Dict[str, Any]:
            """Compare les thématiques entre les documents"""
            comparison = {
                "themes_communs": {},
                "themes_uniques": {},
                "mots_cles_frequence": Counter(),
                "sections_similaires": {},
                "statistiques": {
                    "nombre_documents": len(all_analyses),
                    "themes_par_document": {},
                    "sections_par_document": {}
                }
            }
            
            # Analyser chaque document
            for doc_name, analysis in all_analyses.items():
                themes = analysis["global_themes"]
                comparison["statistiques"]["themes_par_document"][doc_name] = len(themes)
                comparison["statistiques"]["sections_par_document"][doc_name] = len(analysis["sections_analysis"])
                
                # Compter la fréquence des thèmes
                for theme in themes:
                    if theme not in comparison["themes_communs"]:
                        comparison["themes_communs"][theme] = 1
                    else:
                        comparison["themes_communs"][theme] += 1
                
                # Collecter les mots-clés
                comparison["mots_cles_frequence"].update(analysis["all_keywords"])
            
            # Identifier les thèmes uniques et communs
            total_docs = len(all_analyses)
            for theme, count in list(comparison["themes_communs"].items()):
                if count == 1:
                    comparison["themes_uniques"][theme] = count
                    del comparison["themes_communs"][theme]
                else:
                    comparison["themes_communs"][theme] = (count / total_docs) * 100
            
            return comparison

    def generate_comparative_report(self, comparison: Dict[str, Any]) -> str:
            """Génère un rapport comparatif détaillé"""
            report = []
            report.append("📊 ANALYSE COMPARATIVE DES DOCUMENTS")
            report.append(f"Nombre total de documents analysés : {comparison['statistiques']['nombre_documents']}\n")
            
            # Thèmes communs
            report.append("🔄 THÈMES COMMUNS")
            for theme, percentage in sorted(comparison["themes_communs"].items(), key=lambda x: x[1], reverse=True):
                report.append(f"• {theme}: {percentage:.1f}% des documents")
            
            # Thèmes uniques
            report.append("\n💫 THÈMES UNIQUES")
            for theme in comparison["themes_uniques"]:
                report.append(f"• {theme}")
            
            # Mots-clés les plus fréquents
            report.append("\n🔤 MOTS-CLÉS LES PLUS FRÉQUENTS")
            for mot, count in comparison["mots_cles_frequence"].most_common(10):
                report.append(f"• {mot}: {count} occurrences")
            
            # Statistiques par document
            report.append("\n📈 STATISTIQUES PAR DOCUMENT")
            for doc, nb_themes in comparison["statistiques"]["themes_par_document"].items():
                nb_sections = comparison["statistiques"]["sections_par_document"][doc]
                report.append(f"• {doc}:")
                report.append(f"  - Nombre de thèmes : {nb_themes}")
                report.append(f"  - Nombre de sections : {nb_sections}")
            
            return "\n".join(report)

    def analyze_topics(self, all_sections: Dict[str, Dict[str, str]]) -> str:
            """Analyse principale des documents"""
            try:
                print("🔍 Analyse document par document...")
                all_analyses = {}
                
                for doc_name, sections in all_sections.items():
                    print(f"\n📄 Analyse de {doc_name}")
                    doc_analysis = self.analyze_document(sections)
                    all_analyses[doc_name] = doc_analysis
                
                print("\n🔄 Comparaison des documents...")
                comparison = self.compare_documents_themes(all_analyses)
                
                print("\n📊 Génération du rapport...")
                report = self.generate_comparative_report(comparison)
                
                # Sauvegarde des résultats détaillés
                self.save_detailed_results(all_analyses, comparison)
                
                return report
                
            except Exception as e:
                print(f"❌ Erreur analyse: {e}")
                return "Erreur lors de l'analyse"

    def save_detailed_results(self, analyses: Dict, comparison: Dict):
        """Sauvegarde les résultats détaillés"""
        try:
            # Conversion des Counters en dictionnaires
            if "mots_cles_frequence" in comparison:
                comparison["mots_cles_frequence"] = dict(comparison["mots_cles_frequence"])
            
            # Préparation des résultats
            results = {
                "analyses_par_document": {
                    doc_name: {
                        "sections_analysis": doc_analysis["sections_analysis"],
                        "global_themes": doc_analysis["global_themes"],
                        "all_keywords": doc_analysis["all_keywords"]
                    }
                    for doc_name, doc_analysis in analyses.items()
                },
                "comparaison": comparison
            }
            
            with open("analyse_thematique_detaillee.json", "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
                
        except Exception as e:
            print(f"❌ Erreur sauvegarde résultats: {e}")


In [18]:
def main():
    # Initialisation de l'analyseur avec le bon chemin
    analyzer = AAPAnalyzer("../data")  # Changé de "../data" à "data"
    
    print("🔍 Analyse des AAP en cours...")
    # Recherche récursive dans tous les sous-dossiers
    aap_files = []
    for subdir in Path("../data").iterdir():
        if subdir.is_dir():
            aap_files.extend([
                p for p in subdir.rglob("*") 
                if p.is_file() and ("AAP" in p.name.upper() or "aap" in p.name.lower())
            ])
    
    if not aap_files:
        print("❌ Aucun fichier AAP trouvé!")
        return
        
    print(f"📁 {len(aap_files)} fichiers AAP trouvés")
    
    # Vérification de la connexion à Mixtral
    try:
        response = requests.get("http://localhost:11434/api/version")
        if response.status_code != 200:
            print("❌ Erreur: Mixtral n'est pas accessible. Assurez-vous qu'il est lancé sur le port 11434")
            return
    except:
        print("❌ Erreur: Impossible de se connecter à Mixtral. Assurez-vous qu'il est lancé.")
        return
    
    all_sections = {}
    for file_path in aap_files:
        print(f"\n📄 Analyse de {file_path.name}")
        sections = analyzer.extract_text_by_sections(str(file_path))
        if sections:
            all_sections[file_path.name] = sections
    
    if not all_sections:
        print("❌ Aucune section extraite des fichiers!")
        return
    
    print("\n🤖 Analyse thématique en cours...")
    analysis = analyzer.analyze_topics(all_sections)
    
    print("\n=== Rapport d'Analyse ===")
    print(analysis)

# Appel explicite de la fonction main
if __name__ == "__main__":
    main()

🔍 Analyse des AAP en cours...
📁 33 fichiers AAP trouvés

📄 Analyse de ESF_P01_AAP01.docx

📄 Analyse de ESF_P01_AAP02.docx

📄 Analyse de ESF_P01_AAP03.docx

📄 Analyse de ESF_P01_AAP04.docx

📄 Analyse de Guide Pratique Réponse AAP (Groupe SOS).pdf

📄 Analyse de PU_P01_AAP01.docx

📄 Analyse de PU_P01_AAP02.docx

📄 Analyse de PU_P01_AAP03.docx

📄 Analyse de Pulse_P01_AAP01.docx

📄 Analyse de Pulse_P02_AAP01.rtf
⚠️ Format non supporté: ..\data\Pulse\Projet02\Pulse_P02_AAP01.rtf

📄 Analyse de Pulse_P02_AAP01E
⚠️ Format non supporté: ..\data\Pulse\Projet02\Pulse_P02_AAP01E

📄 Analyse de PS_P03_AAP01.docx

📄 Analyse de dossier_de_candidature_aap_projets_innov_entre_2024.docx

📄 Analyse de Fiche candidature AAP jeunes pousses 2024-2026 (2).pdf

📄 Analyse de Note cadrage AAP.docx

📄 Analyse de Objectifs et réalisations AAP jeunes pousses 2024-2026.xlsx

📄 Analyse de Réponse_dossier_de_candidature_aap_projets_innov_entre_2024.pdf

📄 Analyse de Réponse_Fiche candidature AAP jeunes pousses 2024-20

### Futher Approach : plus globale, analyse thematique générale

In [1]:
import requests
import json
from pathlib import Path
import os
from typing import Dict, List, Any, Set, Tuple
from docx import Document
import PyPDF2
import pandas as pd
from collections import Counter, defaultdict
import openpyxl
import logging
from difflib import SequenceMatcher
import numpy as np
from datetime import datetime
import time
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

OLLAMA_HOST = "http://localhost:11434/api/generate"

class AAPAnalyzer:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        self.setup_logging()
        self.theme_patterns = {
            "introduction": ["introduction", "contexte", "présentation", "préambule", "avant-propos"],
            "objectifs": ["objectif", "but", "finalité", "mission", "ambition", "vision"],
            "methodologie": ["méthode", "méthodologie", "approche", "démarche", "processus", "protocole"],
            "budget": ["budget", "financement", "coût", "investissement", "dépense", "ressource financière"],
            "evaluation": ["évaluation", "suivi", "indicateur", "mesure", "performance", "critère"],
            "partenariats": ["partenaire", "collaboration", "coopération", "alliance", "réseau", "acteur"],
            "resultats": ["résultat", "impact", "effet", "retombée", "bénéfice", "conséquence"],
            "planning": ["planning", "calendrier", "échéancier", "temporalité", "délai", "durée"],
            "innovation": ["innovation", "nouveauté", "créativité", "originalité", "invention"],
            "durabilite": ["durabilité", "environnement", "écologie", "développement durable"],
            "social": ["social", "sociétal", "inclusion", "accessibilité", "solidarité"],
            "technique": ["technique", "technologie", "outil", "équipement", "infrastructure"],
            "gouvernance": ["gouvernance", "pilotage", "gestion", "organisation", "coordination"],
            "communication": ["communication", "diffusion", "valorisation", "promotion"]
        }
        
    def setup_logging(self):
        """Configuration des logs avec gestion de l'encodage"""
        if sys.platform == 'win32':
            logging.basicConfig(
                level=logging.INFO,
                format='%(asctime)s - %(levelname)s - %(message)s',
                handlers=[
                    logging.FileHandler('aap_analyzer.log', encoding='utf-8'),
                    logging.StreamHandler(sys.stdout)
                ]
            )
        else:
            logging.basicConfig(
                level=logging.INFO,
                format='%(asctime)s - %(levelname)s - %(message)s',
                handlers=[
                    logging.FileHandler('aap_analyzer.log'),
                    logging.StreamHandler()
                ]
            )
        self.logger = logging.getLogger(__name__)

    def ask_mixtral(self, prompt: str) -> str:
        """Envoie une requête à Mixtral pour analyse et compréhension du texte"""
        try:
            response = requests.post(
                OLLAMA_HOST,
                json={
                    "model": "mixtral",
                    "prompt": prompt,
                    "stream": False,
                    "temperature": 0.3
                },
                timeout=180
            )
            
            if response.status_code == 200:
                return response.json().get("response", "Erreur: pas de réponse")
            else:
                self.logger.error(f"Erreur Mixtral - Status code: {response.status_code}")
                return "Erreur: réponse invalide"
        except Exception as e:
            self.logger.error(f"Erreur connexion Mixtral: {str(e)}")
            return "Erreur: problème de connexion"

    def _extract_from_pdf(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier PDF"""
        sections = {}
        current_section = "Introduction"
        current_content = []
        
        try:
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    content = page.extract_text()
                    lines = content.split('\n')
                    
                    for line in lines:
                        line = line.strip()
                        if not line:
                            continue
                        
                        # Détection d'une nouvelle section
                        if line.startswith('###') or line.isupper():
                            if current_content:
                                sections[current_section] = "\n".join(current_content)
                            current_section = line.strip('#').strip()
                            current_content = []
                        else:
                            current_content.append(line)
            
            # Ajouter la dernière section
            if current_content:
                sections[current_section] = "\n".join(current_content)
                
            return sections
            
        except Exception as e:
            print(f"❌ Erreur PDF {file_path}: {e}")
            return {}

    def _extract_from_excel(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier Excel"""
        sections = {}
        
        try:
            df = pd.read_excel(file_path)
            for col in df.columns:
                if isinstance(col, str) and col.strip():
                    content = df[col].dropna().astype(str).tolist()
                    if content:
                        sections[col] = "\n".join(content)
            return sections
            
        except Exception as e:
            print(f"❌ Erreur Excel {file_path}: {e}")
            return {}
    def extract_text_by_sections(self, file_path: str) -> Dict[str, Any]:
        """Extrait le texte et la structure d'un document"""
        try:
            file_path_lower = file_path.lower()
            if file_path_lower.endswith('.docx'):
                return self._extract_from_docx(file_path)
            elif file_path_lower.endswith('.pdf'):
                return self._extract_from_pdf(file_path)
            elif file_path_lower.endswith(('.xlsx', '.xls')):
                return self._extract_from_excel(file_path)
            else:
                self.logger.warning(f"Format non supporté: {file_path}")
                return {}
        except Exception as e:
            self.logger.error(f"Erreur extraction {file_path}: {str(e)}")
            return {}

    def _extract_from_docx(self, file_path: str) -> Dict[str, Any]:
        """Extraction améliorée des documents Word avec structure"""
        sections = {}
        try:
            doc = Document(file_path)
            current_section = "Introduction"
            sections[current_section] = []
            
            for para in doc.paragraphs:
                text = para.text.strip()
                if not text:
                    continue
                
                if para.style and "Heading" in para.style.name:
                    current_section = text
                    sections[current_section] = []
                else:
                    sections[current_section].append(text)
            
            return {sec: " ".join(content) for sec, content in sections.items() if content}
            
        except Exception as e:
            self.logger.error(f"Erreur Word {file_path}: {str(e)}")
            return {}

    def analyze_themes(self, text: str) -> Dict[str, float]:
        """Analyse les thèmes présents dans un texte"""
        themes_scores = {}
        text_lower = text.lower()
        
        for theme, patterns in self.theme_patterns.items():
            score = 0
            for pattern in patterns:
                if pattern in text_lower:
                    score += text_lower.count(pattern)
            if score > 0:
                themes_scores[theme] = score
                
        # Normalisation des scores
        if themes_scores:
            max_score = max(themes_scores.values())
            themes_scores = {k: v/max_score for k, v in themes_scores.items()}
            
        return themes_scores

    def calculate_similarity(self, text1: str, text2: str) -> float:
        """Calcule la similarité entre deux textes"""
        vectorizer = TfidfVectorizer()
        try:
            tfidf_matrix = vectorizer.fit_transform([text1, text2])
            return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        except:
            return 0.0

    def analyze_aap_files(self) -> Dict[str, Any]:
        """Analyse complète des fichiers AAP"""
        # Vérifier la connexion à Mixtral
        try:
            response = requests.get("http://localhost:11434/api/version")
            if response.status_code != 200:
                self.logger.error("Mixtral n'est pas accessible")
                return {}
        except:
            self.logger.error("Impossible de se connecter à Mixtral")
            return {}

        # Recherche de tous les AAPs dans le dossier data
        aap_files = []
        for path in Path(self.data_dir).rglob("*"):
            if path.is_file() and "AAP" in path.name.upper() and path.suffix.lower() in ['.docx', '.pdf', '.xlsx', '.xls']:
                aap_files.append(path)

        if not aap_files:
            self.logger.error(f"Aucun fichier AAP trouvé dans {self.data_dir}")
            return {}

        self.logger.info(f"Analyse de {len(aap_files)} fichiers AAP")

        # Analyse des documents
        analyses = {}
        processing_times = {}
        similarities = defaultdict(dict)
        total_start_time = time.time()

        # Analyse de chaque AAP
        for aap_path in aap_files:
            self.logger.info(f"Analyse de {aap_path.name}")
            start_time = time.time()

            try:
                # Extraction et analyse des sections
                aap_sections = self.extract_text_by_sections(str(aap_path))
                if not aap_sections:
                    continue

                # Analyse des thèmes par section
                section_themes = {}
                section_similarities = {}
                
                for section, content in aap_sections.items():
                    # Analyse thématique
                    section_themes[section] = self.analyze_themes(content)

                # Construction du prompt pour l'analyse détaillée
                prompt = f"""
                Analyse détaillée du document {aap_path.name}.
                
                Sections trouvées : {list(aap_sections.keys())}
                
                Pour chaque thématique, évalue :
                1. La couverture du contenu (✅ complète, ⚠️ partielle, ❌ absente)
                2. La pertinence du contenu
                3. Les éléments manquants ou à améliorer
                
                Thématiques à analyser :
                {list(self.theme_patterns.keys())}
                
                Format de réponse souhaité :
                - ✅ [Thème] : Bien couvert, avec [détails]
                - ⚠️ [Thème] : Partiellement couvert, manque [éléments]
                - ❌ [Thème] : Absent, devrait inclure [suggestions]
                """

                # Analyse avec Mixtral
                analysis_result = self.ask_mixtral(prompt)

                # Calcul du temps et enregistrement des résultats
                processing_time = time.time() - start_time
                
                analyses[aap_path.name] = {
                    "file_path": str(aap_path),
                    "sections": aap_sections,
                    "themes": section_themes,
                    "analysis": analysis_result,
                    "processing_time": processing_time
                }
                
                processing_times[aap_path.name] = processing_time

            except Exception as e:
                self.logger.error(f"Erreur analyse {aap_path.name}: {str(e)}")
                continue

        total_time = time.time() - total_start_time

        # Génération du rapport
        report = self.generate_report(analyses, processing_times, total_time)

        return {
            "analyses": analyses,
            "processing_times": processing_times,
            "total_time": total_time,
            "report": report
        }

    def analyze_common_themes(self, analyses: Dict[str, Dict]) -> Dict[str, Any]:
        """Analyse les thèmes communs et uniques entre les AAP"""
        # Initialisation des dictionnaires pour stocker les thèmes par document
        themes_by_doc = {}
        all_themes = set()
        
        # Collecte des thèmes pour chaque document
        for doc_name, doc_data in analyses.items():
            doc_themes = set()
            for section_themes in doc_data["themes"].values():
                doc_themes.update(section_themes.keys())
            themes_by_doc[doc_name] = doc_themes
            all_themes.update(doc_themes)
        
        # Identification des thèmes communs à tous les documents
        common_themes = set.intersection(*themes_by_doc.values()) if themes_by_doc else set()
        
        # Identification des thèmes uniques par document
        unique_themes = {}
        for doc_name, themes in themes_by_doc.items():
            other_docs_themes = set.union(*[t for n, t in themes_by_doc.items() if n != doc_name]) if len(themes_by_doc) > 1 else set()
            unique_themes[doc_name] = themes - other_docs_themes
        
        return {
            "common_themes": list(common_themes),
            "unique_themes": {k: list(v) for k, v in unique_themes.items()},
            "all_themes": list(all_themes)
        }

    def generate_report(self, analyses: Dict[str, Dict], processing_times: Dict[str, float], total_time: float) -> str:
        """Génération d'un rapport détaillé"""
        report = []
        report.append("📊 ANALYSE DES AAP")
        report.append(f"Date de l'analyse : {datetime.now().isoformat()}")
        report.append(f"Nombre de documents analysés : {len(analyses)}")
        
        # Analyse comparative des thèmes
        theme_comparison = self.analyze_common_themes(analyses)
        
        report.append("\n🔄 ANALYSE COMPARATIVE DES THÈMES")
        report.append("\n📌 Thèmes communs à tous les AAP :")
        if theme_comparison["common_themes"]:
            for theme in theme_comparison["common_themes"]:
                report.append(f"• {theme}")
        else:
            report.append("Aucun thème commun trouvé")
            
        report.append("\n🎯 Thèmes uniques par AAP :")
        for doc_name, unique in theme_comparison["unique_themes"].items():
            report.append(f"\n{doc_name}:")
            if unique:
                for theme in unique:
                    report.append(f"• {theme}")
            else:
                report.append("Aucun thème unique")
        
        # Temps de traitement
        report.append("\n⏱️ TEMPS DE TRAITEMENT")
        report.append(f"Temps total : {total_time:.2f} secondes")
        report.append("\nTemps par document :")
        for doc, time_spent in processing_times.items():
            report.append(f"• {doc}: {time_spent:.2f} secondes")
        
        # Analyse par document
        report.append("\n📑 ANALYSE PAR DOCUMENT")
        
        for doc_name, doc_data in analyses.items():
            report.append(f"\n=== {doc_name} ===")
            report.append(f"Chemin : {doc_data['file_path']}")
            
            # Sections trouvées
            report.append("\nSections trouvées :")
            for section in doc_data["sections"].keys():
                report.append(f"• {section}")
            
            # Analyse thématique
            report.append("\nAnalyse thématique :")
            for section, themes in doc_data["themes"].items():
                report.append(f"\n• Section : {section}")
                for theme, score in themes.items():
                    report.append(f"  - {theme}: {score:.2f}")
            
            # Analyse détaillée de Mixtral
            report.append("\nAnalyse détaillée :")
            report.append(doc_data["analysis"])
        
        return "\n".join(report)

def check_mixtral():
    """Vérifie si Mixtral est accessible"""
    try:
        response = requests.get("http://localhost:11434/api/version")
        return response.status_code == 200
    except:
        return False

def main():
    print("🔄 Vérification de Mixtral...")
    if not check_mixtral():
        print("❌ Erreur: Mixtral n'est pas accessible. Assurez-vous qu'il est lancé sur le port 11434")
        return

    print("🔍 Initialisation de l'analyse...")
    analyzer = AAPAnalyzer("../data")
    
    try:
        results = analyzer.analyze_aap_files()
        
        if results:
            print("\n=== Rapport d'Analyse ===")
            print(results["report"])
            
            # Sauvegarde des résultats
            with open("analyse_aap_complete.json", "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            
            print("\n💾 Résultats détaillés sauvegardés dans analyse_aap_complete.json")
        else:
            print("\n❌ Aucun résultat d'analyse disponible")
        
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse: {str(e)}")

if __name__ == "__main__":
    main() 

🔄 Vérification de Mixtral...
🔍 Initialisation de l'analyse...
2025-03-11 06:04:05,315 - INFO - Analyse de 31 fichiers AAP
2025-03-11 06:04:05,315 - INFO - Analyse de Guide Pratique Réponse AAP (Groupe SOS).pdf
2025-03-11 06:05:33,033 - INFO - Analyse de ESF_P01_AAP01.docx
2025-03-11 06:05:33,042 - INFO - Analyse de ESF_P01_AAP02.docx
2025-03-11 06:06:44,345 - INFO - Analyse de ESF_P01_AAP03.docx
2025-03-11 06:07:58,154 - INFO - Analyse de ESF_P01_AAP04.docx
2025-03-11 06:09:00,984 - INFO - Analyse de PU_P01_AAP01.docx
2025-03-11 06:12:03,082 - ERROR - Erreur connexion Mixtral: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=180)
2025-03-11 06:12:03,083 - INFO - Analyse de PU_P01_AAP02.docx
2025-03-11 06:13:16,520 - INFO - Analyse de PU_P01_AAP03.docx
2025-03-11 06:16:18,633 - ERROR - Erreur connexion Mixtral: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=180)
2025-03-11 06:16:18,634 - INFO - Analyse de Pulse_P01_AAP01.d

### Based on Guide pratique


In [ ]:
import requests
import json
from pathlib import Path
import os
from typing import Dict, List, Any, Set, Tuple
from docx import Document
import PyPDF2
import pandas as pd
from collections import Counter, defaultdict
import openpyxl
import logging
from difflib import SequenceMatcher
import numpy as np
from datetime import datetime
import time
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

OLLAMA_HOST = "http://localhost:11434/api/generate"

class AAPAnalyzer:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        self.setup_logging()
        self.theme_patterns = {
            "introduction": ["introduction", "contexte", "présentation", "préambule", "avant-propos"],
            "objectifs": ["objectif", "but", "finalité", "mission", "ambition", "vision"],
            "methodologie": ["méthode", "méthodologie", "approche", "démarche", "processus", "protocole"],
            "budget": ["budget", "financement", "coût", "investissement", "dépense", "ressource financière"],
            "evaluation": ["évaluation", "suivi", "indicateur", "mesure", "performance", "critère"],
            "partenariats": ["partenaire", "collaboration", "coopération", "alliance", "réseau", "acteur"],
            "resultats": ["résultat", "impact", "effet", "retombée", "bénéfice", "conséquence"],
            "planning": ["planning", "calendrier", "échéancier", "temporalité", "délai", "durée"],
            "innovation": ["innovation", "nouveauté", "créativité", "originalité", "invention"],
            "durabilite": ["durabilité", "environnement", "écologie", "développement durable"],
            "social": ["social", "sociétal", "inclusion", "accessibilité", "solidarité"],
            "technique": ["technique", "technologie", "outil", "équipement", "infrastructure"],
            "gouvernance": ["gouvernance", "pilotage", "gestion", "organisation", "coordination"],
            "communication": ["communication", "diffusion", "valorisation", "promotion"]
        }
        
        self.reference_sections = {
            "resume": ["résumé", "synthèse", "abstract"],
            "introduction": ["introduction", "contexte", "présentation"],
            "objectifs": ["objectifs", "buts", "finalités"],
            "description_projet": ["description", "présentation détaillée", "contenu"],
            "methodologie": ["méthodologie", "méthode", "approche"],
            "moyens": ["moyens", "ressources", "équipements"],
            "budget": ["budget", "financement", "coûts"],
            "planning": ["planning", "calendrier", "échéancier"],
            "equipe": ["équipe", "personnel", "ressources humaines"],
            "partenaires": ["partenaires", "partenariats", "collaborations"],
            "innovation": ["innovation", "originalité", "nouveauté"],
            "impact": ["impact", "retombées", "effets"],
            "evaluation": ["évaluation", "suivi", "indicateurs"],
            "perennisation": ["pérennisation", "durabilité", "suite"],
            "communication": ["communication", "diffusion", "valorisation"],
            "annexes": ["annexes", "pièces jointes", "documents complémentaires"]
        }

        self.section_requirements = {
            "resume": ["synthèse claire", "objectifs principaux", "résultats attendus"],
            "introduction": ["contexte", "problématique", "enjeux"],
            "objectifs": ["objectifs généraux", "objectifs spécifiques", "résultats attendus"],
            "description_projet": ["description détaillée", "public cible", "territoire"],
            "methodologie": ["approche", "étapes", "outils"],
            "moyens": ["moyens humains", "moyens matériels", "moyens financiers"],
            "budget": ["plan de financement", "coûts détaillés", "sources de financement"],
            "planning": ["calendrier détaillé", "jalons", "livrables"],
            "equipe": ["compétences", "rôles", "responsabilités"],
            "partenaires": ["rôles des partenaires", "engagements", "contributions"],
            "innovation": ["caractère innovant", "plus-value", "originalité"],
            "impact": ["impact social", "impact environnemental", "impact économique"],
            "evaluation": ["indicateurs", "méthodes d'évaluation", "outils de suivi"],
            "perennisation": ["stratégie long terme", "modèle économique", "perspectives"],
            "communication": ["plan de communication", "outils", "cibles"],
            "annexes": ["documents administratifs", "lettres de soutien", "études"]
        }

    def setup_logging(self):
        """Configuration des logs avec gestion de l'encodage"""
        if sys.platform == 'win32':
            logging.basicConfig(
                level=logging.INFO,
                format='%(asctime)s - %(levelname)s - %(message)s',
                handlers=[
                    logging.FileHandler('aap_analyzer.log', encoding='utf-8'),
                    logging.StreamHandler(sys.stdout)
                ]
            )
        else:
            logging.basicConfig(
                level=logging.INFO,
                format='%(asctime)s - %(levelname)s - %(message)s',
                handlers=[
                    logging.FileHandler('aap_analyzer.log'),
                    logging.StreamHandler()
                ]
            )
        self.logger = logging.getLogger(__name__)

    def ask_mixtral(self, prompt: str) -> str:
        """Envoie une requête à Mixtral pour analyse et compréhension du texte"""
        try:
            response = requests.post(
                OLLAMA_HOST,
                json={
                    "model": "mixtral",
                    "prompt": prompt,
                    "stream": False,
                    "temperature": 0.3
                },
                timeout=240
            )
            
            if response.status_code == 200:
                return response.json().get("response", "Erreur: pas de réponse")
            else:
                self.logger.error(f"Erreur Mixtral - Status code: {response.status_code}")
                return "Erreur: réponse invalide"
        except Exception as e:
            self.logger.error(f"Erreur connexion Mixtral: {str(e)}")
            return "Erreur: problème de connexion"
    
    def _extract_from_pdf(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier PDF"""
        sections = {}
        current_section = "Introduction"
        current_content = []
        
        try:
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    content = page.extract_text()
                    lines = content.split('\n')
                    
                    for line in lines:
                        line = line.strip()
                        if not line:
                            continue
                        
                        # Détection d'une nouvelle section
                        if line.startswith('###') or line.isupper():
                            if current_content:
                                sections[current_section] = "\n".join(current_content)
                            current_section = line.strip('#').strip()
                            current_content = []
                        else:
                            current_content.append(line)
            
            # Ajouter la dernière section
            if current_content:
                sections[current_section] = "\n".join(current_content)
                
            return sections
            
        except Exception as e:
            print(f"❌ Erreur PDF {file_path}: {e}")
            return {}

    def _extract_from_excel(self, file_path: str) -> Dict[str, str]:
        """Extrait le texte d'un fichier Excel"""
        sections = {}
        
        try:
            df = pd.read_excel(file_path)
            for col in df.columns:
                if isinstance(col, str) and col.strip():
                    content = df[col].dropna().astype(str).tolist()
                    if content:
                        sections[col] = "\n".join(content)
            return sections
            
        except Exception as e:
            print(f"❌ Erreur Excel {file_path}: {e}")
            return {}

    def extract_text_by_sections(self, file_path: str) -> Dict[str, Any]:
        """Extrait le texte et la structure d'un document"""
        try:
            file_path_lower = file_path.lower()
            if file_path_lower.endswith('.docx'):
                return self._extract_from_docx(file_path)
            elif file_path_lower.endswith('.pdf'):
                return self._extract_from_pdf(file_path)
            elif file_path_lower.endswith(('.xlsx', '.xls')):
                return self._extract_from_excel(file_path)
            else:
                self.logger.warning(f"Format non supporté: {file_path}")
                return {}
        except Exception as e:
            self.logger.error(f"Erreur extraction {file_path}: {str(e)}")
            return {}

    def _extract_from_docx(self, file_path: str) -> Dict[str, Any]:
        """Extraction améliorée des documents Word avec structure"""
        sections = {}
        try:
            doc = Document(file_path)
            current_section = "Introduction"
            sections[current_section] = []
            
            for para in doc.paragraphs:
                text = para.text.strip()
                if not text:
                    continue
                
                if para.style and "Heading" in para.style.name:
                    current_section = text
                    sections[current_section] = []
                else:
                    sections[current_section].append(text)
            
            return {sec: " ".join(content) for sec, content in sections.items() if content}
            
        except Exception as e:
            self.logger.error(f"Erreur Word {file_path}: {str(e)}")
            return {}

    def analyze_themes(self, text: str) -> Dict[str, float]:
        """Analyse les thèmes présents dans un texte"""
        themes_scores = {}
        text_lower = text.lower()
        
        for theme, patterns in self.theme_patterns.items():
            score = 0
            for pattern in patterns:
                if pattern in text_lower:
                    score += text_lower.count(pattern)
            if score > 0:
                themes_scores[theme] = score
                
        # Normalisation des scores
        if themes_scores:
            max_score = max(themes_scores.values())
            themes_scores = {k: v/max_score for k, v in themes_scores.items()}
            
        return themes_scores

    def calculate_similarity(self, text1: str, text2: str) -> float:
        """Calcule la similarité entre deux textes"""
        vectorizer = TfidfVectorizer()
        try:
            tfidf_matrix = vectorizer.fit_transform([text1, text2])
            return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        except:
            return 0.0

    def analyze_aap_files(self) -> Dict[str, Any]:
        """Analyse complète des fichiers AAP"""
        # Vérifier la connexion à Mixtral
        try:
            response = requests.get("http://localhost:11434/api/version")
            if response.status_code != 200:
                self.logger.error("Mixtral n'est pas accessible")
                return {}
        except:
            self.logger.error("Impossible de se connecter à Mixtral")
            return {}

        # Recherche de tous les AAPs dans le dossier data
        aap_files = []
        for path in Path(self.data_dir).rglob("*"):
            if path.is_file() and "AAP" in path.name.upper() and path.suffix.lower() in ['.docx', '.pdf', '.xlsx', '.xls']:
                aap_files.append(path)

        if not aap_files:
            self.logger.error(f"Aucun fichier AAP trouvé dans {self.data_dir}")
            return {}

        self.logger.info(f"Analyse de {len(aap_files)} fichiers AAP")

        # Analyse des documents
        analyses = {}
        processing_times = {}
        similarities = defaultdict(dict)
        total_start_time = time.time()

        # Analyse de chaque AAP
        for aap_path in aap_files:
            self.logger.info(f"Analyse de {aap_path.name}")
            start_time = time.time()

            try:
                # Extraction et analyse des sections
                aap_sections = self.extract_text_by_sections(str(aap_path))
                if not aap_sections:
                    continue

                # Analyse des thèmes par section
                section_themes = {}
                section_similarities = {}
                
                for section, content in aap_sections.items():
                    # Analyse thématique
                    section_themes[section] = self.analyze_themes(content)

                # Construction du prompt pour l'analyse détaillée
                prompt = f"""
                Analyse détaillée du document {aap_path.name}.
                
                Sections trouvées : {list(aap_sections.keys())}
                
                Pour chaque thématique, évalue :
                1. La couverture du contenu (✅ complète, ⚠️ partielle, ❌ absente)
                2. La pertinence du contenu
                3. Les éléments manquants ou à améliorer
                
                Thématiques à analyser :
                {list(self.theme_patterns.keys())}
                
                Format de réponse souhaité :
                - ✅ [Thème] : Bien couvert, avec [détails]
                - ⚠️ [Thème] : Partiellement couvert, manque [éléments]
                - ❌ [Thème] : Absent, devrait inclure [suggestions]
                """

                # Analyse avec Mixtral
                analysis_result = self.ask_mixtral(prompt)

                # Calcul du temps et enregistrement des résultats
                processing_time = time.time() - start_time
                
                analyses[aap_path.name] = {
                    "file_path": str(aap_path),
                    "sections": aap_sections,
                    "themes": section_themes,
                    "analysis": analysis_result,
                    "processing_time": processing_time
                }
                
                processing_times[aap_path.name] = processing_time

            except Exception as e:
                self.logger.error(f"Erreur analyse {aap_path.name}: {str(e)}")
                continue

        total_time = time.time() - total_start_time

        # Génération du rapport
        report = self.generate_report(analyses, processing_times, total_time)

        return {
            "analyses": analyses,
            "processing_times": processing_times,
            "total_time": total_time,
            "report": report
        }

    def analyze_common_themes(self, analyses: Dict[str, Dict]) -> Dict[str, Any]:
        """Analyse les thèmes communs et uniques entre les AAP"""
        # Initialisation des dictionnaires pour stocker les thèmes par document
        themes_by_doc = {}
        all_themes = set()
        
        # Collecte des thèmes pour chaque document
        for doc_name, doc_data in analyses.items():
            doc_themes = set()
            for section_themes in doc_data["themes"].values():
                doc_themes.update(section_themes.keys())
            themes_by_doc[doc_name] = doc_themes
            all_themes.update(doc_themes)
        
        # Identification des thèmes communs à tous les documents
        common_themes = set.intersection(*themes_by_doc.values()) if themes_by_doc else set()
        
        # Identification des thèmes uniques par document
        unique_themes = {}
        for doc_name, themes in themes_by_doc.items():
            other_docs_themes = set.union(*[t for n, t in themes_by_doc.items() if n != doc_name]) if len(themes_by_doc) > 1 else set()
            unique_themes[doc_name] = themes - other_docs_themes
        
        return {
            "common_themes": list(common_themes),
            "unique_themes": {k: list(v) for k, v in unique_themes.items()},
            "all_themes": list(all_themes)
        }

    def analyze_section_completeness(self, section_content: str, section_name: str) -> Dict[str, Any]:
        """Analyse la complétude d'une section par rapport aux exigences"""
        requirements = self.section_requirements.get(section_name, [])
        completeness = {}
        
        for req in requirements:
            # Calcul du score de présence pour chaque exigence
            score = sum(1 for word in req.split() if word.lower() in section_content.lower())
            completeness[req] = {
                "present": score > 0,
                "score": score / len(req.split())  # Score normalisé
            }
            
        return {
            "requirements_met": sum(1 for req in completeness.values() if req["present"]),
            "total_requirements": len(requirements),
            "details": completeness,
            "completion_rate": sum(req["score"] for req in completeness.values()) / len(requirements) if requirements else 0
        }

    def analyze_aap_statistics(self, analyses: Dict[str, Dict]) -> Dict[str, Any]:
        """Génère des statistiques détaillées pour chaque AAP"""
        stats = {}
        
        for doc_name, doc_data in analyses.items():
            doc_stats = {
                "section_coverage": {},
                "theme_coverage": {},
                "global_stats": {
                    "total_sections": len(doc_data["sections"]),
                    "total_themes": len(doc_data["themes"]),
                    "average_completion": 0
                }
            }
            
            # Analyse de la couverture des sections
            total_completion = 0
            for section_name, content in doc_data["sections"].items():
                completeness = self.analyze_section_completeness(content, section_name)
                doc_stats["section_coverage"][section_name] = completeness
                total_completion += completeness["completion_rate"]
            
            # Calcul des moyennes
            doc_stats["global_stats"]["average_completion"] = (
                total_completion / len(doc_data["sections"]) if doc_data["sections"] else 0
            )
            
            # Analyse des thèmes
            for section, themes in doc_data["themes"].items():
                for theme, score in themes.items():
                    if theme not in doc_stats["theme_coverage"]:
                        doc_stats["theme_coverage"][theme] = {
                            "total_score": 0,
                            "occurrences": 0
                        }
                    doc_stats["theme_coverage"][theme]["total_score"] += score
                    doc_stats["theme_coverage"][theme]["occurrences"] += 1
            
            # Calcul des scores moyens des thèmes
            for theme_stats in doc_stats["theme_coverage"].values():
                theme_stats["average_score"] = (
                    theme_stats["total_score"] / theme_stats["occurrences"]
                    if theme_stats["occurrences"] > 0 else 0
                )
            
            stats[doc_name] = doc_stats
        
        return stats

    def generate_report(self, analyses: Dict[str, Dict], processing_times: Dict[str, float], total_time: float) -> str:
        """Génération d'un rapport détaillé"""
        report = []
        report.append("📊 ANALYSE DES AAP")
        report.append(f"Date de l'analyse : {datetime.now().isoformat()}")
        report.append(f"Nombre de documents analysés : {len(analyses)}")
        
        # Analyse comparative des thèmes
        theme_comparison = self.analyze_common_themes(analyses)
        
        # Statistiques détaillées
        stats = self.analyze_aap_statistics(analyses)
        
        report.append("\n📈 STATISTIQUES GLOBALES")
        for doc_name, doc_stats in stats.items():
            report.append(f"\n=== {doc_name} ===")
            report.append(f"Taux de complétion moyen : {doc_stats['global_stats']['average_completion']*100:.1f}%")
            report.append(f"Nombre de sections : {doc_stats['global_stats']['total_sections']}")
            
            report.append("\nCouverture des sections :")
            for section, coverage in doc_stats["section_coverage"].items():
                completion = coverage["completion_rate"] * 100
                requirements_met = coverage["requirements_met"]
                total_req = coverage["total_requirements"]
                
                if completion >= 75:
                    status = "✅"
                elif completion >= 40:
                    status = "⚠️"
                else:
                    status = "❌"
                
                report.append(f"{status} {section}: {completion:.1f}% ({requirements_met}/{total_req} critères)")
                
                # Détails des exigences non satisfaites
                missing_reqs = [
                    req for req, details in coverage["details"].items()
                    if not details["present"]
                ]
                if missing_reqs:
                    report.append("   Éléments manquants :")
                    for req in missing_reqs:
                        report.append(f"   - {req}")
            
            report.append("\nCouverture thématique :")
            for theme, coverage in doc_stats["theme_coverage"].items():
                score = coverage["average_score"] * 100
                if score >= 75:
                    status = "✅"
                elif score >= 40:
                    status = "⚠️"
                else:
                    status = "❌"
                report.append(f"{status} {theme}: {score:.1f}%")
        
        report.append("\n🔄 ANALYSE COMPARATIVE DES THÈMES")
        report.append("\n📌 Thèmes communs à tous les AAP :")
        if theme_comparison["common_themes"]:
            for theme in theme_comparison["common_themes"]:
                report.append(f"• {theme}")
        else:
            report.append("Aucun thème commun trouvé")
            
        report.append("\n🎯 Thèmes uniques par AAP :")
        for doc_name, unique in theme_comparison["unique_themes"].items():
            report.append(f"\n{doc_name}:")
            if unique:
                for theme in unique:
                    report.append(f"• {theme}")
            else:
                report.append("Aucun thème unique")
        
        # Temps de traitement
        report.append("\n⏱️ TEMPS DE TRAITEMENT")
        report.append(f"Temps total : {total_time:.2f} secondes")
        report.append("\nTemps par document :")
        for doc, time_spent in processing_times.items():
            report.append(f"• {doc}: {time_spent:.2f} secondes")
        
        return "\n".join(report)

def check_mixtral():
    """Vérifie si Mixtral est accessible"""
    try:
        response = requests.get("http://localhost:11434/api/version")
        return response.status_code == 200
    except:
        return False

def main():
    print("🔄 Vérification de Mixtral...")
    if not check_mixtral():
        print("❌ Erreur: Mixtral n'est pas accessible. Assurez-vous qu'il est lancé sur le port 11434")
        return

    print("🔍 Initialisation de l'analyse...")
    analyzer = AAPAnalyzer("../data")
    
    try:
        results = analyzer.analyze_aap_files()
        
        if results:
            print("\n=== Rapport d'Analyse ===")
            print(results["report"])
            
            # Sauvegarde des résultats
            with open("analyse_aap_complete.json", "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            
            print("\n💾 Résultats détaillés sauvegardés dans analyse_aap_complete.json")
        else:
            print("\n❌ Aucun résultat d'analyse disponible")
        
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse: {str(e)}")

if __name__ == "__main__":
    main() 

🔄 Vérification de Mixtral...
🔍 Initialisation de l'analyse...
2025-03-11 09:15:22,729 - INFO - Analyse de 31 fichiers AAP
2025-03-11 09:15:22,730 - INFO - Analyse de Guide Pratique Réponse AAP (Groupe SOS).pdf
